# evaluating effects of missing data threshold and population assignment on PCA, t-SNE, and dxy 

In [5]:
import ipyrad.analysis as ipa
import toyplot
import ipyparallel as ipp
import pandas as pd
import numpy as np
from sklearn.manifold import TSNE

In [6]:
# the path to your HDF5 formatted snps file
data = "../plate1/M_ru_3rm_v9_outfiles/M_ru_3rm_v9.snps.hdf5"

Let's define populations Western Inambari (Inam), Purus-Madeira (Puru), Jiparana-Guapore (JiGu), Jiparaná-Roosevelt (Mach), Roosevelt-Aripuana (Roar), Aripuana-Sucunduri (ArSu), Sucunduri-Tapajos (SuTa), and Western Para (Para). 

These assignments are made only to color points in the PCA analysis below.

In [7]:
imap = {
#"ref": ["reference"],
"Inam": ['M_ru_A7875_In','M_ru_T14456_In', 'M_ru_T23245_In', 'M_ru_T23246_In', 'M_ru_T23416_In', 'M_ru_T23478_In'],
"Puru": ['M_ru_A10311_pu','M_ru_A10329_pu', 'M_ru_A1380_pu', 'M_ru_A2741_pu', 'M_ru_A436_pu', 'M_ru_A440_pu'],
"JiGu": ['M_ru_A207_jigu', 'M_ru_T3228_jigu', 'M_ru_T7634_jigu'],
"Mach": ['M_ru_T13253_ma','M_ru_A474_ma', 'M_ru_T3164_ma', 'M_ru_J265_ma', 'M_ru_T368_ma', 'M_ru_T381_ma', 'M_ru_T476_ma', 'M_ru_T494_ma'],
"Roar": ['M_ru_J640_roar','M_ru_J676_roar'],
"ArSu": ['M_ru_80819_arsu', 'M_ru_81347_arsu', 'M_ru_85426_arsu'],
"SuTa": [ 'M_ru_85919_suta','M_ru_77750_suta', 'M_ru_78182_suta', 'M_ru_A11834_suta', 'M_ru_A15176_suta', 'M_ru_A5487_suta', 'M_ru_T10184_suta', 'M_ru_T11780_suta', 'M_ru_T14532_suta', 'M_ru_T14622_suta', 'M_ru_T753_suta', 'M_ru_A9903_pa'],
"Para": ['M_ru_A16195_pa','M_ru_A9235_pa', 'M_ru_T11079_pa', 'M_ru_T11238_pa', 'M_ru_T12541_pa', 'M_ru_T1649_pa', 'M_ru_T16553_pa', 'M_ru_T19782_pa', 'M_ru_T6500_pa', 'M_ru_T6577_pa']
}
# minimum % of samples that must be present in each SNP from each population: do 5 subsets to see robustness of impu
#Because there is a lot of missing data, we want to make sure that results are not biased due to imputing too many haplotypes
#sampling across different minmaps will help address this. If the data are more organized by population with more missing data, then this could be bias in the imputation from small sample sizes
minmap1 = {i: 0.5 for i in imap}
minmap2 = {i: 0.65 for i in imap}
minmap3 = {i: 0.75 for i in imap}
minmap4 = {i: 0.85 for i in imap}
minmap5 = {i: 0.95 for i in imap}

let's run PCA for different values of minmap to see how varying missing data affects our results

note we are assuming K=8 which is likely high, but k-means clustering will cluster samples independent of a priori geographic assignment

Here we are using k-means clustering to assign individuals to populations independently of our a priori geographic assignments in order to circumscribe populations from which to impute missing haplotypes.

The PCA plots can then be colored based on our geographic assignments above

In [8]:
#init pca object with input data and (optional) parameter options

pca = ipa.pca(
    data=data,
    imap=imap,
    minmap=minmap1,
    mincov=0.85,
    impute_method=8,
)
pca2 = ipa.pca(
    data=data,
    imap=imap,
    minmap=minmap2,
    mincov=0.85,
    impute_method=8,
)
pca3 = ipa.pca(
    data=data,
    imap=imap,
    minmap=minmap3,
    mincov=0.85,
    impute_method=8,
)
pca4 = ipa.pca(
    data=data,
    imap=imap,
    minmap=minmap4,
    mincov=0.85,
    impute_method=8,
)
pca5 = ipa.pca(
    data=data,
    imap=imap,
    minmap=minmap5,
    mincov=0.85,
    impute_method=8,
)

Kmeans clustering: iter=0, K=8, mincov=0.9, minmap={'global': 0.85}
Samples: 50
Sites before filtering: 819934
Filtered (indels): 0
Filtered (bi-allel): 14244
Filtered (mincov): 691215
Filtered (minmap): 666808
Filtered (combined): 694543
Sites after filtering: 125391
Sites containing missing values: 92803 (74.01%)
Missing values in SNP matrix: 243340 (3.88%)
Imputation: 'sampled'; (0, 1, 2) = 35.8%, 3.4%, 60.8%
{0: ['M_ru_A10311_pu', 'M_ru_A10329_pu', 'M_ru_A1380_pu', 'M_ru_A2741_pu', 'M_ru_A436_pu', 'M_ru_A440_pu'], 1: ['M_ru_77750_suta', 'M_ru_78182_suta', 'M_ru_80819_arsu', 'M_ru_81347_arsu', 'M_ru_85426_arsu', 'M_ru_85919_suta', 'M_ru_A11834_suta', 'M_ru_A15176_suta', 'M_ru_A5487_suta', 'M_ru_A9903_pa', 'M_ru_J640_roar', 'M_ru_T10184_suta', 'M_ru_T11780_suta', 'M_ru_T14532_suta', 'M_ru_T14622_suta', 'M_ru_T753_suta'], 2: ['M_ru_A16195_pa', 'M_ru_A9235_pa', 'M_ru_T11079_pa', 'M_ru_T11238_pa', 'M_ru_T12541_pa', 'M_ru_T1649_pa', 'M_ru_T16553_pa', 'M_ru_T19782_pa', 'M_ru_T6500_pa', 'M

In [9]:
# # run the PCA analysis
pca.run()
pca2.run()
pca3.run()
pca4.run()
pca5.run()

Subsampling SNPs: 6711/48914
Subsampling SNPs: 6780/49618
Subsampling SNPs: 6681/48637
Subsampling SNPs: 6873/50442
Subsampling SNPs: 4482/32588


In [10]:
pca.draw()

(<toyplot.canvas.Canvas at 0x2b4ea43a1850>,
 <toyplot.coordinates.Cartesian at 0x2b4ea43a1450>)

<svg class="toyplot-canvas-Canvas" height="300.0px" id="te5909f05ef2d4b60998e40a2a7dd1084" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> M_ru_77750_suta M_ru_78182_suta M_ru_80819_arsu M_ru_81347_arsu M_ru_85426_arsu M_ru_85919_suta M_ru_A10311_pu M_ru_A10329_pu M_ru_A11834_suta M_ru_A1380_pu M_ru_A15176_suta M_ru_A16195_pa M_ru_A207_jigu M_ru_A2741_pu M_ru_A436_pu M_ru_A440_pu M_ru_A474_ma M_ru_A5487_suta M_ru_A7875_In M_ru_A9235_pa M_ru_A9903_pa M_ru_J265_ma M_ru_J640_roar M_ru_J676_roar M_ru_T10184_suta M_ru_T11079_pa M_ru_T11238_pa M_ru_T11780_suta M_ru_T12541_pa M_ru_T13253_ma M_ru_T14456_In M_ru_T14532_suta M_ru_T14622_suta M_ru_T1649_pa M_ru_T16553_pa M_ru_T19782_pa M_ru_T23245_In M_ru_T23246_In M_ru_T23416_In M_ru_T23478_In M_ru_T3164_ma M_ru_T3228_jigu M_ru_T368_ma M_ru_T381_ma M_ru_T476_ma M_ru_T494_ma M_ru_T6500_pa M_ru_T6577_pa M_ru_T753_suta M_ru_T7634_jigu -10 0 10 PC0 (27.4%) explained -5 0 5 10 15 PC1 (17.4%) explained Inam Puru JiGu Mach Roar ArSu SuTa Para

In [11]:
pca2.draw()

(<toyplot.canvas.Canvas at 0x2b4ea31f4c50>,
 <toyplot.coordinates.Cartesian at 0x2b4ea31f4650>)

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t60e87fec7ad146789b46f6b06f6af048" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> M_ru_77750_suta M_ru_78182_suta M_ru_80819_arsu M_ru_81347_arsu M_ru_85426_arsu M_ru_85919_suta M_ru_A10311_pu M_ru_A10329_pu M_ru_A11834_suta M_ru_A1380_pu M_ru_A15176_suta M_ru_A16195_pa M_ru_A207_jigu M_ru_A2741_pu M_ru_A436_pu M_ru_A440_pu M_ru_A474_ma M_ru_A5487_suta M_ru_A7875_In M_ru_A9235_pa M_ru_A9903_pa M_ru_J265_ma M_ru_J640_roar M_ru_J676_roar M_ru_T10184_suta M_ru_T11079_pa M_ru_T11238_pa M_ru_T11780_suta M_ru_T12541_pa M_ru_T13253_ma M_ru_T14456_In M_ru_T14532_suta M_ru_T14622_suta M_ru_T1649_pa M_ru_T16553_pa M_ru_T19782_pa M_ru_T23245_In M_ru_T23246_In M_ru_T23416_In M_ru_T23478_In M_ru_T3164_ma M_ru_T3228_jigu M_ru_T368_ma M_ru_T381_ma M_ru_T476_ma M_ru_T494_ma M_ru_T6500_pa M_ru_T6577_pa M_ru_T753_suta M_ru_T7634_jigu -10 0 10 PC0 (26.1%) explained -5 0 5 10 15 PC1 (20.1%) explained Inam Puru JiGu Mach Roar ArSu SuTa Para

In [12]:
pca3.draw()

(<toyplot.canvas.Canvas at 0x2b4ea354f750>,
 <toyplot.coordinates.Cartesian at 0x2b4ea354f890>)

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t307db367404a409dafb1f3b3bf9c7b68" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> M_ru_77750_suta M_ru_78182_suta M_ru_80819_arsu M_ru_81347_arsu M_ru_85426_arsu M_ru_85919_suta M_ru_A10311_pu M_ru_A10329_pu M_ru_A11834_suta M_ru_A1380_pu M_ru_A15176_suta M_ru_A16195_pa M_ru_A207_jigu M_ru_A2741_pu M_ru_A436_pu M_ru_A440_pu M_ru_A474_ma M_ru_A5487_suta M_ru_A7875_In M_ru_A9235_pa M_ru_A9903_pa M_ru_J265_ma M_ru_J640_roar M_ru_J676_roar M_ru_T10184_suta M_ru_T11079_pa M_ru_T11238_pa M_ru_T11780_suta M_ru_T12541_pa M_ru_T13253_ma M_ru_T14456_In M_ru_T14532_suta M_ru_T14622_suta M_ru_T1649_pa M_ru_T16553_pa M_ru_T19782_pa M_ru_T23245_In M_ru_T23246_In M_ru_T23416_In M_ru_T23478_In M_ru_T3164_ma M_ru_T3228_jigu M_ru_T368_ma M_ru_T381_ma M_ru_T476_ma M_ru_T494_ma M_ru_T6500_pa M_ru_T6577_pa M_ru_T753_suta M_ru_T7634_jigu -10 0 10 PC0 (28.4%) explained -5 0 5 10 15 PC1 (18.1%) explained Inam Puru JiGu Mach Roar ArSu SuTa Para

In [13]:
pca4.draw()

(<toyplot.canvas.Canvas at 0x2b4ea3561350>,
 <toyplot.coordinates.Cartesian at 0x2b4ea35613d0>)

<svg class="toyplot-canvas-Canvas" height="300.0px" id="taf6c9bfcc6c748f28d2bb98e62686f01" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> M_ru_77750_suta M_ru_78182_suta M_ru_80819_arsu M_ru_81347_arsu M_ru_85426_arsu M_ru_85919_suta M_ru_A10311_pu M_ru_A10329_pu M_ru_A11834_suta M_ru_A1380_pu M_ru_A15176_suta M_ru_A16195_pa M_ru_A207_jigu M_ru_A2741_pu M_ru_A436_pu M_ru_A440_pu M_ru_A474_ma M_ru_A5487_suta M_ru_A7875_In M_ru_A9235_pa M_ru_A9903_pa M_ru_J265_ma M_ru_J640_roar M_ru_J676_roar M_ru_T10184_suta M_ru_T11079_pa M_ru_T11238_pa M_ru_T11780_suta M_ru_T12541_pa M_ru_T13253_ma M_ru_T14456_In M_ru_T14532_suta M_ru_T14622_suta M_ru_T1649_pa M_ru_T16553_pa M_ru_T19782_pa M_ru_T23245_In M_ru_T23246_In M_ru_T23416_In M_ru_T23478_In M_ru_T3164_ma M_ru_T3228_jigu M_ru_T368_ma M_ru_T381_ma M_ru_T476_ma M_ru_T494_ma M_ru_T6500_pa M_ru_T6577_pa M_ru_T753_suta M_ru_T7634_jigu -10 0 10 PC0 (26.9%) explained -5 0 5 10 15 PC1 (17.9%) explained Inam Puru JiGu Mach Roar ArSu SuTa Para

In [14]:
pca5.draw()

(<toyplot.canvas.Canvas at 0x2b4ea355f150>,
 <toyplot.coordinates.Cartesian at 0x2b4ea355fcd0>)

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t39a37e18a4e9416799087f60e57023a9" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> M_ru_77750_suta M_ru_78182_suta M_ru_80819_arsu M_ru_81347_arsu M_ru_85426_arsu M_ru_85919_suta M_ru_A10311_pu M_ru_A10329_pu M_ru_A11834_suta M_ru_A1380_pu M_ru_A15176_suta M_ru_A16195_pa M_ru_A207_jigu M_ru_A2741_pu M_ru_A436_pu M_ru_A440_pu M_ru_A474_ma M_ru_A5487_suta M_ru_A7875_In M_ru_A9235_pa M_ru_A9903_pa M_ru_J265_ma M_ru_J640_roar M_ru_J676_roar M_ru_T10184_suta M_ru_T11079_pa M_ru_T11238_pa M_ru_T11780_suta M_ru_T12541_pa M_ru_T13253_ma M_ru_T14456_In M_ru_T14532_suta M_ru_T14622_suta M_ru_T1649_pa M_ru_T16553_pa M_ru_T19782_pa M_ru_T23245_In M_ru_T23246_In M_ru_T23416_In M_ru_T23478_In M_ru_T3164_ma M_ru_T3228_jigu M_ru_T368_ma M_ru_T381_ma M_ru_T476_ma M_ru_T494_ma M_ru_T6500_pa M_ru_T6577_pa M_ru_T753_suta M_ru_T7634_jigu -5 0 5 10 PC0 (26.1%) explained -5 0 5 10 PC1 (19.7%) explained Inam Puru JiGu Mach Roar ArSu SuTa Para

As you can see, for varying degrees of missing data, we generally get the same or similar results.

there are three to five clusters of points here that correspond pretty clearly to many river barriers and these are consistent among runs.

Now we can write the PCA results to a file

In [15]:
# # store the PC axes as a dataframe
df4 = pd.DataFrame(pca4.pcaxes[0], index=pca4.names)

# # write the PC axes to a CSV file
df4.to_csv("M_ru_pca_85minmap_12Jan2022.csv")

# # show the first ten samples and the first 10 PC axes
df4.iloc[:10, :10].round(2)

,0,1,2,3,4,5,6,7,8,9
M_ru_77750_suta,-7.83,-6.14,0.75,-0.07,-0.43,-1.63,-0.09,-2.10,-0.72,-0.17
M_ru_78182_suta,-8.18,-6.12,0.44,-0.54,-0.50,-2.00,-2.12,-3.96,-5.10,4.03
M_ru_80819_arsu,-7.65,-5.85,0.70,-0.28,-1.13,-2.84,-1.65,-2.65,5.15,-3.56
M_ru_81347_arsu,-8.33,-6.14,0.43,-0.19,0.08,-1.30,-1.20,-1.75,-1.22,3.33
M_ru_85426_arsu,-7.77,-5.50,0.36,-0.54,-0.53,-1.90,0.47,-0.28,-0.08,-0.43
M_ru_85919_suta,-7.63,-5.35,0.13,-0.33,-0.61,-1.78,-0.96,-0.72,-2.58,0.08
M_ru_A10311_pu,11.39,-0.85,-7.66,-6.94,-0.70,-0.09,-0.30,-0.29,1.32,1.22
M_ru_A10329_pu,11.58,-0.68,-7.18,-7.26,-0.71,-0.48,0.11,-2.17,2.92,1.64
M_ru_A11834_suta,-8.00,-6.31,0.46,0.27,-0.17,-2.94,0.24,0.74,0.68,0.52
M_ru_A1380_pu,11.71,-0.67,-6.95,-6.64,-0.22,0.31,-0.40,1.67,-0.72,1.44


Now let's look at if and how t-SNE differs from PCA

In [16]:
pca.run_tsne(subsample=True, perplexity=5.0, n_iter=1000000, seed=123)
pca2.run_tsne(subsample=True, perplexity=5.0, n_iter=1000000, seed=123)
pca3.run_tsne(subsample=True, perplexity=5.0, n_iter=1000000, seed=123)
pca4.run_tsne(subsample=True, perplexity=5.0, n_iter=10000000, seed=123)
pca5.run_tsne(subsample=True, perplexity=5.0, n_iter=1000000, seed=223)

Subsampling SNPs: 6711/48914
Subsampling SNPs: 6780/49618
Subsampling SNPs: 6681/48637
Subsampling SNPs: 6873/50442
Subsampling SNPs: 4482/32588


In [17]:
pca.draw();
pca2.draw();
pca3.draw();
pca4.draw();
pca5.draw();

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t689fc3c3c5d441fdaff4ab38bdc2d944" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> M_ru_77750_suta M_ru_78182_suta M_ru_80819_arsu M_ru_81347_arsu M_ru_85426_arsu M_ru_85919_suta M_ru_A10311_pu M_ru_A10329_pu M_ru_A11834_suta M_ru_A1380_pu M_ru_A15176_suta M_ru_A16195_pa M_ru_A207_jigu M_ru_A2741_pu M_ru_A436_pu M_ru_A440_pu M_ru_A474_ma M_ru_A5487_suta M_ru_A7875_In M_ru_A9235_pa M_ru_A9903_pa M_ru_J265_ma M_ru_J640_roar M_ru_J676_roar M_ru_T10184_suta M_ru_T11079_pa M_ru_T11238_pa M_ru_T11780_suta M_ru_T12541_pa M_ru_T13253_ma M_ru_T14456_In M_ru_T14532_suta M_ru_T14622_suta M_ru_T1649_pa M_ru_T16553_pa M_ru_T19782_pa M_ru_T23245_In M_ru_T23246_In M_ru_T23416_In M_ru_T23478_In M_ru_T3164_ma M_ru_T3228_jigu M_ru_T368_ma M_ru_T381_ma M_ru_T476_ma M_ru_T494_ma M_ru_T6500_pa M_ru_T6577_pa M_ru_T753_suta M_ru_T7634_jigu -100 0 100 200 TSNE component 1 -3000 -2000 -1000 0 TSNE component 2 Inam Puru JiGu Mach Roar ArSu SuTa Para

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t72ef252c832a4045bdb3259f1aa6fd9e" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> M_ru_77750_suta M_ru_78182_suta M_ru_80819_arsu M_ru_81347_arsu M_ru_85426_arsu M_ru_85919_suta M_ru_A10311_pu M_ru_A10329_pu M_ru_A11834_suta M_ru_A1380_pu M_ru_A15176_suta M_ru_A16195_pa M_ru_A207_jigu M_ru_A2741_pu M_ru_A436_pu M_ru_A440_pu M_ru_A474_ma M_ru_A5487_suta M_ru_A7875_In M_ru_A9235_pa M_ru_A9903_pa M_ru_J265_ma M_ru_J640_roar M_ru_J676_roar M_ru_T10184_suta M_ru_T11079_pa M_ru_T11238_pa M_ru_T11780_suta M_ru_T12541_pa M_ru_T13253_ma M_ru_T14456_In M_ru_T14532_suta M_ru_T14622_suta M_ru_T1649_pa M_ru_T16553_pa M_ru_T19782_pa M_ru_T23245_In M_ru_T23246_In M_ru_T23416_In M_ru_T23478_In M_ru_T3164_ma M_ru_T3228_jigu M_ru_T368_ma M_ru_T381_ma M_ru_T476_ma M_ru_T494_ma M_ru_T6500_pa M_ru_T6577_pa M_ru_T753_suta M_ru_T7634_jigu -500 0 500 TSNE component 1 -2500 0 2500 5000 7500 TSNE component 2 Inam Puru JiGu Mach Roar ArSu SuTa Para

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t06d8dcf7b18e4ef896d6e2fd4939dbcc" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> M_ru_77750_suta M_ru_78182_suta M_ru_80819_arsu M_ru_81347_arsu M_ru_85426_arsu M_ru_85919_suta M_ru_A10311_pu M_ru_A10329_pu M_ru_A11834_suta M_ru_A1380_pu M_ru_A15176_suta M_ru_A16195_pa M_ru_A207_jigu M_ru_A2741_pu M_ru_A436_pu M_ru_A440_pu M_ru_A474_ma M_ru_A5487_suta M_ru_A7875_In M_ru_A9235_pa M_ru_A9903_pa M_ru_J265_ma M_ru_J640_roar M_ru_J676_roar M_ru_T10184_suta M_ru_T11079_pa M_ru_T11238_pa M_ru_T11780_suta M_ru_T12541_pa M_ru_T13253_ma M_ru_T14456_In M_ru_T14532_suta M_ru_T14622_suta M_ru_T1649_pa M_ru_T16553_pa M_ru_T19782_pa M_ru_T23245_In M_ru_T23246_In M_ru_T23416_In M_ru_T23478_In M_ru_T3164_ma M_ru_T3228_jigu M_ru_T368_ma M_ru_T381_ma M_ru_T476_ma M_ru_T494_ma M_ru_T6500_pa M_ru_T6577_pa M_ru_T753_suta M_ru_T7634_jigu -100 0 100 200 TSNE component 1 -200 -100 0 100 200 TSNE component 2 Inam Puru JiGu Mach Roar ArSu SuTa Para

<svg class="toyplot-canvas-Canvas" height="300.0px" id="ta8fc9dc062fd43788e5775951e2978ca" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> M_ru_77750_suta M_ru_78182_suta M_ru_80819_arsu M_ru_81347_arsu M_ru_85426_arsu M_ru_85919_suta M_ru_A10311_pu M_ru_A10329_pu M_ru_A11834_suta M_ru_A1380_pu M_ru_A15176_suta M_ru_A16195_pa M_ru_A207_jigu M_ru_A2741_pu M_ru_A436_pu M_ru_A440_pu M_ru_A474_ma M_ru_A5487_suta M_ru_A7875_In M_ru_A9235_pa M_ru_A9903_pa M_ru_J265_ma M_ru_J640_roar M_ru_J676_roar M_ru_T10184_suta M_ru_T11079_pa M_ru_T11238_pa M_ru_T11780_suta M_ru_T12541_pa M_ru_T13253_ma M_ru_T14456_In M_ru_T14532_suta M_ru_T14622_suta M_ru_T1649_pa M_ru_T16553_pa M_ru_T19782_pa M_ru_T23245_In M_ru_T23246_In M_ru_T23416_In M_ru_T23478_In M_ru_T3164_ma M_ru_T3228_jigu M_ru_T368_ma M_ru_T381_ma M_ru_T476_ma M_ru_T494_ma M_ru_T6500_pa M_ru_T6577_pa M_ru_T753_suta M_ru_T7634_jigu -1000 -500 0 500 TSNE component 1 -1500 -1000 -500 0 500 TSNE component 2 Inam Puru JiGu Mach Roar ArSu SuTa Para

<svg class="toyplot-canvas-Canvas" height="300.0px" id="td0ec118262214c429364279e0d491170" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> M_ru_77750_suta M_ru_78182_suta M_ru_80819_arsu M_ru_81347_arsu M_ru_85426_arsu M_ru_85919_suta M_ru_A10311_pu M_ru_A10329_pu M_ru_A11834_suta M_ru_A1380_pu M_ru_A15176_suta M_ru_A16195_pa M_ru_A207_jigu M_ru_A2741_pu M_ru_A436_pu M_ru_A440_pu M_ru_A474_ma M_ru_A5487_suta M_ru_A7875_In M_ru_A9235_pa M_ru_A9903_pa M_ru_J265_ma M_ru_J640_roar M_ru_J676_roar M_ru_T10184_suta M_ru_T11079_pa M_ru_T11238_pa M_ru_T11780_suta M_ru_T12541_pa M_ru_T13253_ma M_ru_T14456_In M_ru_T14532_suta M_ru_T14622_suta M_ru_T1649_pa M_ru_T16553_pa M_ru_T19782_pa M_ru_T23245_In M_ru_T23246_In M_ru_T23416_In M_ru_T23478_In M_ru_T3164_ma M_ru_T3228_jigu M_ru_T368_ma M_ru_T381_ma M_ru_T476_ma M_ru_T494_ma M_ru_T6500_pa M_ru_T6577_pa M_ru_T753_suta M_ru_T7634_jigu -200 -100 0 100 TSNE component 1 -100 -50 0 50 TSNE component 2 Inam Puru JiGu Mach Roar ArSu SuTa Para

amazingly with t-sne we see significantly more clusters

note that these results are sensitive to values of perplexity and starting seed, so you can play with those parameters to get more interpretable results

Then we can again write the t-sne results to a file

In [18]:
# # store the PC axes as a dataframe
df4 = pd.DataFrame(pca4.pcaxes[0], index=pca4.names)

# # write the PC axes to a CSV file
df4.to_csv("M_ru_TSNE_85minmap_12Jan2022.csv")

# # show the first ten samples and the first 10 PC axes
df4.iloc[:10, :10].round(2)

,0,1
M_ru_77750_suta,-155.21,81.63
M_ru_78182_suta,-123.90,124.63
M_ru_80819_arsu,-142.68,92.18
M_ru_81347_arsu,-158.49,119.06
M_ru_85426_arsu,-124.72,98.65
M_ru_85919_suta,-92.67,102.26
M_ru_A10311_pu,137.12,-20.80
M_ru_A10329_pu,133.07,-42.80
M_ru_A11834_suta,-94.39,59.35
M_ru_A1380_pu,176.64,-20.26


Let's skip this for now, but the next section of code does 10,000 TSNE replicates for downstream analysis using randomply generated values for starting seed and "perplexity"

In [19]:
# !rm -r M_ru_TSNE
# !mkdir M_ru_TSNE
# import random
# for i in range(10000):
#     pca3.run_tsne(subsample=True, perplexity=random.randrange(3,8), n_iter=100000, seed=random.randrange(100,9999))
#     df4 = pd.DataFrame(pca3.pcaxes[0], index=pca4.names)
#     df4.to_csv("./M_ru_TSNE/M_ru_TSNE_rep"+str(i)+".csv")

#### Now let's look at just the western clade, and here we assume K=4; again clustering is independent of geography

In [20]:
#RUN AGAIN WITH WESTERN POPULATIONS ONLY

imap = {
#"ref": ["reference"],
"Inam": ['M_ru_A7875_In','M_ru_T14456_In', 'M_ru_T23245_In', 'M_ru_T23246_In', 'M_ru_T23416_In', 'M_ru_T23478_In'],
"Puru": ['M_ru_A10311_pu','M_ru_A10329_pu', 'M_ru_A1380_pu', 'M_ru_A2741_pu', 'M_ru_A436_pu', 'M_ru_A440_pu'],
"JiGu": ['M_ru_A207_jigu', 'M_ru_T3228_jigu', 'M_ru_T7634_jigu'],
"Mach": ['M_ru_T13253_ma','M_ru_A474_ma', 'M_ru_T3164_ma', 'M_ru_J265_ma'],#'M_ru_T368_ma', 'M_ru_T381_ma', 'M_ru_T476_ma', 'M_ru_T494_ma'
#"Roar": ['M_ru_J640_roar','M_ru_J676_roar'],
#"ArSu": ['M_ru_80819_arsu', 'M_ru_81347_arsu', 'M_ru_85426_arsu'],
#"SuTa": [ 'M_ru_85919_suta','M_ru_77750_suta', 'M_ru_78182_suta', 'M_ru_A11834_suta', 'M_ru_A15176_suta', 'M_ru_A5487_suta', 'M_ru_T10184_suta', 'M_ru_T11780_suta', 'M_ru_T14532_suta', 'M_ru_T14622_suta', 'M_ru_T753_suta', 'M_ru_A9903_pa'],
#"Para": ['M_ru_A16195_pa','M_ru_A9235_pa', 'M_ru_T11079_pa', 'M_ru_T11238_pa', 'M_ru_T12541_pa', 'M_ru_T1649_pa', 'M_ru_T16553_pa', 'M_ru_T19782_pa', 'M_ru_T6500_pa', 'M_ru_T6577_pa']
}

# minimum % of samples that must be present in each SNP from each group
minmap1 = {i: 0.55 for i in imap}
minmap2 = {i: 0.65 for i in imap}
minmap3 = {i: 0.75 for i in imap}
minmap4 = {i: 0.85 for i in imap}
minmap5 = {i: 0.95 for i in imap}

In [21]:
# init pca object with input data and (optional) parameter options
pca = ipa.pca(
    data=data,
    imap=imap,
    minmap=minmap1,
    mincov=0.85,
    impute_method=4,
)
# init pca object with input data and (optional) parameter options
pca2 = ipa.pca(
    data=data,
    imap=imap,
    minmap=minmap2,
    mincov=0.85,
    impute_method=4,
)
# init pca object with input data and (optional) parameter options
pca3 = ipa.pca(
    data=data,
    imap=imap,
    minmap=minmap3,
    mincov=0.85,
    impute_method=4,
)
# init pca object with input data and (optional) parameter options
pca4 = ipa.pca(
    data=data,
    imap=imap,
    minmap=minmap4,
    mincov=0.85,
    impute_method=4,
)
# init pca object with input data and (optional) parameter options
pca5 = ipa.pca(
    data=data,
    imap=imap,
    minmap=minmap5,
    mincov=0.85,
    impute_method=4,
)

Kmeans clustering: iter=0, K=4, mincov=0.9, minmap={'global': 0.85}
Samples: 19
Sites before filtering: 819934
Filtered (indels): 0
Filtered (bi-allel): 6174
Filtered (mincov): 710465
Filtered (minmap): 675930
Filtered (combined): 711703
Sites after filtering: 108231
Sites containing missing values: 42638 (39.40%)
Missing values in SNP matrix: 42638 (2.07%)
Imputation: 'sampled'; (0, 1, 2) = 35.2%, 2.5%, 62.2%
{0: ['M_ru_A10311_pu', 'M_ru_A10329_pu', 'M_ru_A1380_pu', 'M_ru_A2741_pu', 'M_ru_A436_pu', 'M_ru_A440_pu'], 1: ['M_ru_A7875_In', 'M_ru_T14456_In', 'M_ru_T23245_In', 'M_ru_T23246_In', 'M_ru_T23416_In', 'M_ru_T23478_In'], 2: ['M_ru_A474_ma', 'M_ru_J265_ma', 'M_ru_T13253_ma', 'M_ru_T3164_ma'], 3: ['M_ru_A207_jigu', 'M_ru_T3228_jigu', 'M_ru_T7634_jigu']}

Kmeans clustering: iter=1, K=4, mincov=0.8875, minmap={0: 0.85, 1: 0.85, 2: 0.85, 3: 0.85}
Samples: 19
Sites before filtering: 819934
Filtered (indels): 0
Filtered (bi-allel): 6174
Filtered (mincov): 675930
Filtered (minmap): 753598

/home/lmusher/nas5/miniconda3/lib/python3.7/site-packages/ipyrad/analysis/snps_imputer.py:118: RuntimeWarning: invalid value encountered in long_scalars
  100 * np.sum(imputed == 0) / imputed.size,
/home/lmusher/nas5/miniconda3/lib/python3.7/site-packages/ipyrad/analysis/snps_imputer.py:119: RuntimeWarning: invalid value encountered in long_scalars
  100 * np.sum(imputed == 1) / imputed.size,
/home/lmusher/nas5/miniconda3/lib/python3.7/site-packages/ipyrad/analysis/snps_imputer.py:120: RuntimeWarning: invalid value encountered in long_scalars
  100 * np.sum(imputed == 2) / imputed.size,


{0: ['M_ru_A207_jigu', 'M_ru_T3228_jigu', 'M_ru_T7634_jigu'], 1: ['M_ru_A7875_In', 'M_ru_T14456_In', 'M_ru_T23245_In', 'M_ru_T23246_In', 'M_ru_T23416_In', 'M_ru_T23478_In'], 2: ['M_ru_A10311_pu', 'M_ru_A10329_pu', 'M_ru_A1380_pu', 'M_ru_A2741_pu', 'M_ru_A436_pu', 'M_ru_A440_pu'], 3: ['M_ru_A474_ma', 'M_ru_J265_ma', 'M_ru_T13253_ma', 'M_ru_T3164_ma']}

Kmeans clustering: iter=2, K=4, mincov=0.875, minmap={0: 0.85, 1: 0.85, 2: 0.85, 3: 0.85}
Samples: 19
Sites before filtering: 819934
Filtered (indels): 0
Filtered (bi-allel): 6174
Filtered (mincov): 675930
Filtered (minmap): 753598
Filtered (combined): 754341
Sites after filtering: 65593
Sites containing missing values: 0 (0.00%)
Missing values in SNP matrix: 0 (0.00%)
Imputation: 'sampled'; (0, 1, 2) = nan%, nan%, nan%


/home/lmusher/nas5/miniconda3/lib/python3.7/site-packages/ipyrad/analysis/snps_imputer.py:118: RuntimeWarning: invalid value encountered in long_scalars
  100 * np.sum(imputed == 0) / imputed.size,
/home/lmusher/nas5/miniconda3/lib/python3.7/site-packages/ipyrad/analysis/snps_imputer.py:119: RuntimeWarning: invalid value encountered in long_scalars
  100 * np.sum(imputed == 1) / imputed.size,
/home/lmusher/nas5/miniconda3/lib/python3.7/site-packages/ipyrad/analysis/snps_imputer.py:120: RuntimeWarning: invalid value encountered in long_scalars
  100 * np.sum(imputed == 2) / imputed.size,


{0: ['M_ru_A474_ma', 'M_ru_J265_ma', 'M_ru_T13253_ma', 'M_ru_T3164_ma'], 1: ['M_ru_A7875_In', 'M_ru_T14456_In', 'M_ru_T23245_In', 'M_ru_T23246_In', 'M_ru_T23416_In', 'M_ru_T23478_In'], 2: ['M_ru_A10311_pu', 'M_ru_A10329_pu', 'M_ru_A1380_pu', 'M_ru_A2741_pu', 'M_ru_A436_pu', 'M_ru_A440_pu'], 3: ['M_ru_A207_jigu', 'M_ru_T3228_jigu', 'M_ru_T7634_jigu']}

Kmeans clustering: iter=3, K=4, mincov=0.8625, minmap={0: 0.85, 1: 0.85, 2: 0.85, 3: 0.85}
Samples: 19
Sites before filtering: 819934
Filtered (indels): 0
Filtered (bi-allel): 6174
Filtered (mincov): 675930
Filtered (minmap): 753598
Filtered (combined): 754341
Sites after filtering: 65593
Sites containing missing values: 0 (0.00%)
Missing values in SNP matrix: 0 (0.00%)
Imputation: 'sampled'; (0, 1, 2) = nan%, nan%, nan%


...

/home/lmusher/nas5/miniconda3/lib/python3.7/site-packages/ipyrad/analysis/snps_imputer.py:118: RuntimeWarning: invalid value encountered in long_scalars
  100 * np.sum(imputed == 0) / imputed.size,
/home/lmusher/nas5/miniconda3/lib/python3.7/site-packages/ipyrad/analysis/snps_imputer.py:119: RuntimeWarning: invalid value encountered in long_scalars
  100 * np.sum(imputed == 1) / imputed.size,
/home/lmusher/nas5/miniconda3/lib/python3.7/site-packages/ipyrad/analysis/snps_imputer.py:120: RuntimeWarning: invalid value encountered in long_scalars
  100 * np.sum(imputed == 2) / imputed.size,


{0: ['M_ru_A10311_pu', 'M_ru_A10329_pu', 'M_ru_A1380_pu', 'M_ru_A2741_pu', 'M_ru_A436_pu', 'M_ru_A440_pu'], 1: ['M_ru_A7875_In', 'M_ru_T14456_In', 'M_ru_T23245_In', 'M_ru_T23246_In', 'M_ru_T23416_In', 'M_ru_T23478_In'], 2: ['M_ru_A474_ma', 'M_ru_J265_ma', 'M_ru_T13253_ma', 'M_ru_T3164_ma'], 3: ['M_ru_A207_jigu', 'M_ru_T3228_jigu', 'M_ru_T7634_jigu']}

Kmeans clustering: iter=3, K=4, mincov=0.8625, minmap={0: 0.85, 1: 0.85, 2: 0.85, 3: 0.85}
Samples: 19
Sites before filtering: 819934
Filtered (indels): 0
Filtered (bi-allel): 6174
Filtered (mincov): 675930
Filtered (minmap): 753598
Filtered (combined): 754341
Sites after filtering: 65593
Sites containing missing values: 0 (0.00%)
Missing values in SNP matrix: 0 (0.00%)
Imputation: 'sampled'; (0, 1, 2) = nan%, nan%, nan%


/home/lmusher/nas5/miniconda3/lib/python3.7/site-packages/ipyrad/analysis/snps_imputer.py:118: RuntimeWarning: invalid value encountered in long_scalars
  100 * np.sum(imputed == 0) / imputed.size,
/home/lmusher/nas5/miniconda3/lib/python3.7/site-packages/ipyrad/analysis/snps_imputer.py:119: RuntimeWarning: invalid value encountered in long_scalars
  100 * np.sum(imputed == 1) / imputed.size,
/home/lmusher/nas5/miniconda3/lib/python3.7/site-packages/ipyrad/analysis/snps_imputer.py:120: RuntimeWarning: invalid value encountered in long_scalars
  100 * np.sum(imputed == 2) / imputed.size,


{0: ['M_ru_A7875_In', 'M_ru_T14456_In', 'M_ru_T23245_In', 'M_ru_T23246_In', 'M_ru_T23416_In', 'M_ru_T23478_In'], 1: ['M_ru_A10311_pu', 'M_ru_A10329_pu', 'M_ru_A1380_pu', 'M_ru_A2741_pu', 'M_ru_A436_pu', 'M_ru_A440_pu'], 2: ['M_ru_A474_ma', 'M_ru_J265_ma', 'M_ru_T13253_ma', 'M_ru_T3164_ma'], 3: ['M_ru_A207_jigu', 'M_ru_T3228_jigu', 'M_ru_T7634_jigu']}

Kmeans clustering: iter=4, K=4, mincov=0.85, minmap={0: 0.85, 1: 0.85, 2: 0.85, 3: 0.85}
Samples: 19
Sites before filtering: 819934
Filtered (indels): 0
Filtered (bi-allel): 6174
Filtered (mincov): 675930
Filtered (minmap): 753598
Filtered (combined): 754341
Sites after filtering: 65593
Sites containing missing values: 0 (0.00%)
Missing values in SNP matrix: 0 (0.00%)
Imputation: 'sampled'; (0, 1, 2) = nan%, nan%, nan%


/home/lmusher/nas5/miniconda3/lib/python3.7/site-packages/ipyrad/analysis/snps_imputer.py:118: RuntimeWarning: invalid value encountered in long_scalars
  100 * np.sum(imputed == 0) / imputed.size,
/home/lmusher/nas5/miniconda3/lib/python3.7/site-packages/ipyrad/analysis/snps_imputer.py:119: RuntimeWarning: invalid value encountered in long_scalars
  100 * np.sum(imputed == 1) / imputed.size,
/home/lmusher/nas5/miniconda3/lib/python3.7/site-packages/ipyrad/analysis/snps_imputer.py:120: RuntimeWarning: invalid value encountered in long_scalars
  100 * np.sum(imputed == 2) / imputed.size,


In [22]:
# run the PCA analysis
pca.run()
pca2.run()
pca3.run()
pca4.run()
pca5.run()

Subsampling SNPs: 8454/65593
Subsampling SNPs: 8454/65593
Subsampling SNPs: 8454/65593
Subsampling SNPs: 8454/65593
Subsampling SNPs: 8454/65593


In [23]:
pca.draw()
pca2.draw()
pca3.draw()
pca4.draw()
pca5.draw()

(<toyplot.canvas.Canvas at 0x2b4ea70ddd10>,
 <toyplot.coordinates.Cartesian at 0x2b4ea70dd950>)

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t1e5c36391b4345d8ad95249c07c89678" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> M_ru_A10311_pu M_ru_A10329_pu M_ru_A1380_pu M_ru_A207_jigu M_ru_A2741_pu M_ru_A436_pu M_ru_A440_pu M_ru_A474_ma M_ru_A7875_In M_ru_J265_ma M_ru_T13253_ma M_ru_T14456_In M_ru_T23245_In M_ru_T23246_In M_ru_T23416_In M_ru_T23478_In M_ru_T3164_ma M_ru_T3228_jigu M_ru_T7634_jigu -8 0 8 16 PC0 (35.4%) explained -10 -5 0 5 10 PC1 (13.9%) explained Inam Puru JiGu Mach

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t52a9b877c7a34a9996289205ccddbfe9" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> M_ru_A10311_pu M_ru_A10329_pu M_ru_A1380_pu M_ru_A207_jigu M_ru_A2741_pu M_ru_A436_pu M_ru_A440_pu M_ru_A474_ma M_ru_A7875_In M_ru_J265_ma M_ru_T13253_ma M_ru_T14456_In M_ru_T23245_In M_ru_T23246_In M_ru_T23416_In M_ru_T23478_In M_ru_T3164_ma M_ru_T3228_jigu M_ru_T7634_jigu -10 -5 0 5 10 15 PC0 (34.5%) explained -10 0 10 PC1 (14.0%) explained Inam Puru JiGu Mach

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t60e0c0db61de4bc3aa9d836670515f2e" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> M_ru_A10311_pu M_ru_A10329_pu M_ru_A1380_pu M_ru_A207_jigu M_ru_A2741_pu M_ru_A436_pu M_ru_A440_pu M_ru_A474_ma M_ru_A7875_In M_ru_J265_ma M_ru_T13253_ma M_ru_T14456_In M_ru_T23245_In M_ru_T23246_In M_ru_T23416_In M_ru_T23478_In M_ru_T3164_ma M_ru_T3228_jigu M_ru_T7634_jigu -10 -5 0 5 10 15 PC0 (33.4%) explained -10 -5 0 5 10 PC1 (13.8%) explained Inam Puru JiGu Mach

<svg class="toyplot-canvas-Canvas" height="300.0px" id="tb465dc4442d94d6fa50c6f77ee726afb" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> M_ru_A10311_pu M_ru_A10329_pu M_ru_A1380_pu M_ru_A207_jigu M_ru_A2741_pu M_ru_A436_pu M_ru_A440_pu M_ru_A474_ma M_ru_A7875_In M_ru_J265_ma M_ru_T13253_ma M_ru_T14456_In M_ru_T23245_In M_ru_T23246_In M_ru_T23416_In M_ru_T23478_In M_ru_T3164_ma M_ru_T3228_jigu M_ru_T7634_jigu -5 0 5 10 15 PC0 (35.0%) explained -10 -5 0 5 10 PC1 (14.8%) explained Inam Puru JiGu Mach

<svg class="toyplot-canvas-Canvas" height="300.0px" id="tf223cb7b9450482890fb3c81a38b3c41" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> M_ru_A10311_pu M_ru_A10329_pu M_ru_A1380_pu M_ru_A207_jigu M_ru_A2741_pu M_ru_A436_pu M_ru_A440_pu M_ru_A474_ma M_ru_A7875_In M_ru_J265_ma M_ru_T13253_ma M_ru_T14456_In M_ru_T23245_In M_ru_T23246_In M_ru_T23416_In M_ru_T23478_In M_ru_T3164_ma M_ru_T3228_jigu M_ru_T7634_jigu -8 0 8 16 PC0 (33.4%) explained -10 -5 0 5 10 PC1 (14.1%) explained Inam Puru JiGu Mach

When we zoom in on one clade, we can see that there is even more structure than we initially thought

In [24]:
pca.run_tsne(subsample=True, perplexity=5.0, n_iter=1000000, seed=123)
pca2.run_tsne(subsample=True, perplexity=5.0, n_iter=1000000, seed=123)
pca3.run_tsne(subsample=True, perplexity=5.0, n_iter=1000000, seed=123)
pca4.run_tsne(subsample=True, perplexity=5.0, n_iter=1000000, seed=123)
pca5.run_tsne(subsample=True, perplexity=5.0, n_iter=1000000, seed=123)

Subsampling SNPs: 8454/65593
Subsampling SNPs: 8454/65593
Subsampling SNPs: 8454/65593
Subsampling SNPs: 8454/65593
Subsampling SNPs: 8454/65593


In [25]:
pca.draw();
pca2.draw();
pca3.draw();
pca4.draw();
pca5.draw();

<svg class="toyplot-canvas-Canvas" height="300.0px" id="tc4cf0e9dfdb0409f9e2b517d40c09e5f" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> M_ru_A10311_pu M_ru_A10329_pu M_ru_A1380_pu M_ru_A207_jigu M_ru_A2741_pu M_ru_A436_pu M_ru_A440_pu M_ru_A474_ma M_ru_A7875_In M_ru_J265_ma M_ru_T13253_ma M_ru_T14456_In M_ru_T23245_In M_ru_T23246_In M_ru_T23416_In M_ru_T23478_In M_ru_T3164_ma M_ru_T3228_jigu M_ru_T7634_jigu -200 -100 0 100 TSNE component 1 -200 0 200 400 TSNE component 2 Inam Puru JiGu Mach

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t12e1bf287e8b4aa184a82d707d866767" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> M_ru_A10311_pu M_ru_A10329_pu M_ru_A1380_pu M_ru_A207_jigu M_ru_A2741_pu M_ru_A436_pu M_ru_A440_pu M_ru_A474_ma M_ru_A7875_In M_ru_J265_ma M_ru_T13253_ma M_ru_T14456_In M_ru_T23245_In M_ru_T23246_In M_ru_T23416_In M_ru_T23478_In M_ru_T3164_ma M_ru_T3228_jigu M_ru_T7634_jigu -200 -100 0 100 TSNE component 1 -200 0 200 400 TSNE component 2 Inam Puru JiGu Mach

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t0a48edbc89944d34b7b6263892489e4b" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> M_ru_A10311_pu M_ru_A10329_pu M_ru_A1380_pu M_ru_A207_jigu M_ru_A2741_pu M_ru_A436_pu M_ru_A440_pu M_ru_A474_ma M_ru_A7875_In M_ru_J265_ma M_ru_T13253_ma M_ru_T14456_In M_ru_T23245_In M_ru_T23246_In M_ru_T23416_In M_ru_T23478_In M_ru_T3164_ma M_ru_T3228_jigu M_ru_T7634_jigu -200 -100 0 100 TSNE component 1 -200 0 200 400 TSNE component 2 Inam Puru JiGu Mach

<svg class="toyplot-canvas-Canvas" height="300.0px" id="tb376e51c575b4a2bbc5d4b56afe99ab4" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> M_ru_A10311_pu M_ru_A10329_pu M_ru_A1380_pu M_ru_A207_jigu M_ru_A2741_pu M_ru_A436_pu M_ru_A440_pu M_ru_A474_ma M_ru_A7875_In M_ru_J265_ma M_ru_T13253_ma M_ru_T14456_In M_ru_T23245_In M_ru_T23246_In M_ru_T23416_In M_ru_T23478_In M_ru_T3164_ma M_ru_T3228_jigu M_ru_T7634_jigu -200 -100 0 100 TSNE component 1 -200 0 200 400 TSNE component 2 Inam Puru JiGu Mach

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t9633ec19c75e4b439b21551e68768cbe" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> M_ru_A10311_pu M_ru_A10329_pu M_ru_A1380_pu M_ru_A207_jigu M_ru_A2741_pu M_ru_A436_pu M_ru_A440_pu M_ru_A474_ma M_ru_A7875_In M_ru_J265_ma M_ru_T13253_ma M_ru_T14456_In M_ru_T23245_In M_ru_T23246_In M_ru_T23416_In M_ru_T23478_In M_ru_T3164_ma M_ru_T3228_jigu M_ru_T7634_jigu -200 -100 0 100 TSNE component 1 -200 0 200 400 TSNE component 2 Inam Puru JiGu Mach

Now let's look at just the eastern clade, and here we assume K=5; again clustering is independent of geography

In [26]:
#RUN AGAIN WITH EASTERN POPULATIONS ONLY

imap = {
#"ref": ["reference"],
#"Inam": ['M_ru_A7875_In','M_ru_T14456_In', 'M_ru_T23245_In', 'M_ru_T23246_In', 'M_ru_T23416_In', 'M_ru_T23478_In'],
#"Puru": ['M_ru_A10311_pu','M_ru_A10329_pu', 'M_ru_A1380_pu', 'M_ru_A2741_pu', 'M_ru_A436_pu', 'M_ru_A440_pu'],
#"JiGu": ['M_ru_A207_jigu', 'M_ru_T3228_jigu', 'M_ru_T7634_jigu'],
"Mach": ['M_ru_T368_ma', 'M_ru_T381_ma', 'M_ru_T476_ma', 'M_ru_T494_ma'],
"Roar": ['M_ru_J640_roar','M_ru_J676_roar'],
"ArSu": ['M_ru_80819_arsu', 'M_ru_81347_arsu', 'M_ru_85426_arsu'],
"SuTa": [ 'M_ru_85919_suta','M_ru_77750_suta', 'M_ru_78182_suta', 'M_ru_A11834_suta', 'M_ru_A15176_suta', 'M_ru_A5487_suta', 'M_ru_T10184_suta', 'M_ru_T11780_suta', 'M_ru_T14532_suta', 'M_ru_T14622_suta', 'M_ru_T753_suta', 'M_ru_A9903_pa'],
"Para": ['M_ru_A16195_pa','M_ru_A9235_pa', 'M_ru_T11079_pa', 'M_ru_T11238_pa', 'M_ru_T12541_pa', 'M_ru_T1649_pa', 'M_ru_T16553_pa', 'M_ru_T19782_pa', 'M_ru_T6500_pa', 'M_ru_T6577_pa']
}

# minimum % of samples that must be present in each SNP from each group
minmap1 = {i: 0.55 for i in imap}
minmap2 = {i: 0.65 for i in imap}
minmap3 = {i: 0.75 for i in imap}
minmap4 = {i: 0.85 for i in imap}
minmap5 = {i: 0.95 for i in imap}

In [27]:
# init pca object with input data and (optional) parameter options
pca = ipa.pca(
    data=data,
    imap=imap,
    minmap=minmap1,
    mincov=0.85,
    impute_method=5,
)
# init pca object with input data and (optional) parameter options
pca2 = ipa.pca(
    data=data,
    imap=imap,
    minmap=minmap2,
    mincov=0.85,
    impute_method=5,
)
# init pca object with input data and (optional) parameter options
pca3 = ipa.pca(
    data=data,
    imap=imap,
    minmap=minmap3,
    mincov=0.85,
    impute_method=5,
)
# init pca object with input data and (optional) parameter options
pca4 = ipa.pca(
    data=data,
    imap=imap,
    minmap=minmap4,
    mincov=0.85,
    impute_method=5,
)
# init pca object with input data and (optional) parameter options
pca5 = ipa.pca(
    data=data,
    imap=imap,
    minmap=minmap5,
    mincov=0.85,
    impute_method=5,
)

Kmeans clustering: iter=0, K=5, mincov=0.9, minmap={'global': 0.85}
Samples: 31
Sites before filtering: 819934
Filtered (indels): 0
Filtered (bi-allel): 9343
Filtered (mincov): 675574
Filtered (minmap): 656833
Filtered (combined): 678027
Sites after filtering: 141907
Sites containing missing values: 92999 (65.54%)
Missing values in SNP matrix: 163650 (3.72%)
Imputation: 'sampled'; (0, 1, 2) = 35.3%, 2.8%, 61.9%
{0: ['M_ru_A16195_pa', 'M_ru_A9235_pa', 'M_ru_T11238_pa', 'M_ru_T12541_pa', 'M_ru_T1649_pa', 'M_ru_T16553_pa', 'M_ru_T19782_pa', 'M_ru_T6500_pa', 'M_ru_T6577_pa'], 1: ['M_ru_85426_arsu', 'M_ru_J676_roar', 'M_ru_T10184_suta', 'M_ru_T11780_suta', 'M_ru_T14622_suta'], 2: ['M_ru_77750_suta', 'M_ru_78182_suta', 'M_ru_80819_arsu', 'M_ru_81347_arsu', 'M_ru_85919_suta', 'M_ru_A11834_suta', 'M_ru_A15176_suta', 'M_ru_A5487_suta', 'M_ru_A9903_pa', 'M_ru_J640_roar', 'M_ru_T14532_suta', 'M_ru_T753_suta'], 3: ['M_ru_T368_ma', 'M_ru_T381_ma', 'M_ru_T476_ma', 'M_ru_T494_ma'], 4: ['M_ru_T11079_p

In [28]:
# run the PCA analysis
pca.run()
pca2.run()
pca3.run()
pca4.run()
pca5.run()

Subsampling SNPs: 13732/114554
Subsampling SNPs: 11711/92842
Subsampling SNPs: 12743/103830
Subsampling SNPs: 13723/114754
Subsampling SNPs: 6264/48908


In [29]:
pca.draw()
pca2.draw()
pca3.draw()
pca4.draw()
pca5.draw()

(<toyplot.canvas.Canvas at 0x2b4eaaa52a50>,
 <toyplot.coordinates.Cartesian at 0x2b4eaaa52b10>)

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t9c9540cef6014203b40120468e9e450f" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> M_ru_77750_suta M_ru_78182_suta M_ru_80819_arsu M_ru_81347_arsu M_ru_85426_arsu M_ru_85919_suta M_ru_A11834_suta M_ru_A15176_suta M_ru_A16195_pa M_ru_A5487_suta M_ru_A9235_pa M_ru_A9903_pa M_ru_J640_roar M_ru_J676_roar M_ru_T10184_suta M_ru_T11079_pa M_ru_T11238_pa M_ru_T11780_suta M_ru_T12541_pa M_ru_T14532_suta M_ru_T14622_suta M_ru_T1649_pa M_ru_T16553_pa M_ru_T19782_pa M_ru_T368_ma M_ru_T381_ma M_ru_T476_ma M_ru_T494_ma M_ru_T6500_pa M_ru_T6577_pa M_ru_T753_suta -10 0 10 20 PC0 (43.9%) explained -5 0 5 10 PC1 (4.6%) explained Mach Roar ArSu SuTa Para

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t7e02d156797f4adfa54afb1673bb562b" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> M_ru_77750_suta M_ru_78182_suta M_ru_80819_arsu M_ru_81347_arsu M_ru_85426_arsu M_ru_85919_suta M_ru_A11834_suta M_ru_A15176_suta M_ru_A16195_pa M_ru_A5487_suta M_ru_A9235_pa M_ru_A9903_pa M_ru_J640_roar M_ru_J676_roar M_ru_T10184_suta M_ru_T11079_pa M_ru_T11238_pa M_ru_T11780_suta M_ru_T12541_pa M_ru_T14532_suta M_ru_T14622_suta M_ru_T1649_pa M_ru_T16553_pa M_ru_T19782_pa M_ru_T368_ma M_ru_T381_ma M_ru_T476_ma M_ru_T494_ma M_ru_T6500_pa M_ru_T6577_pa M_ru_T753_suta -10 0 10 20 PC0 (43.3%) explained -5 0 5 10 PC1 (4.9%) explained Mach Roar ArSu SuTa Para

<svg class="toyplot-canvas-Canvas" height="300.0px" id="tfdfa333a6b3c4c269dcb8b4290cf0ac6" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> M_ru_77750_suta M_ru_78182_suta M_ru_80819_arsu M_ru_81347_arsu M_ru_85426_arsu M_ru_85919_suta M_ru_A11834_suta M_ru_A15176_suta M_ru_A16195_pa M_ru_A5487_suta M_ru_A9235_pa M_ru_A9903_pa M_ru_J640_roar M_ru_J676_roar M_ru_T10184_suta M_ru_T11079_pa M_ru_T11238_pa M_ru_T11780_suta M_ru_T12541_pa M_ru_T14532_suta M_ru_T14622_suta M_ru_T1649_pa M_ru_T16553_pa M_ru_T19782_pa M_ru_T368_ma M_ru_T381_ma M_ru_T476_ma M_ru_T494_ma M_ru_T6500_pa M_ru_T6577_pa M_ru_T753_suta -10 0 10 20 PC0 (41.3%) explained -5 0 5 10 PC1 (5.2%) explained Mach Roar ArSu SuTa Para

<svg class="toyplot-canvas-Canvas" height="300.0px" id="td04441f25bbe41cea62faa81bc810fa6" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> M_ru_77750_suta M_ru_78182_suta M_ru_80819_arsu M_ru_81347_arsu M_ru_85426_arsu M_ru_85919_suta M_ru_A11834_suta M_ru_A15176_suta M_ru_A16195_pa M_ru_A5487_suta M_ru_A9235_pa M_ru_A9903_pa M_ru_J640_roar M_ru_J676_roar M_ru_T10184_suta M_ru_T11079_pa M_ru_T11238_pa M_ru_T11780_suta M_ru_T12541_pa M_ru_T14532_suta M_ru_T14622_suta M_ru_T1649_pa M_ru_T16553_pa M_ru_T19782_pa M_ru_T368_ma M_ru_T381_ma M_ru_T476_ma M_ru_T494_ma M_ru_T6500_pa M_ru_T6577_pa M_ru_T753_suta -10 0 10 20 PC0 (43.4%) explained -5 0 5 10 15 PC1 (4.9%) explained Mach Roar ArSu SuTa Para

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t278419da38fa4f85a8b63d39c21b1006" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> M_ru_77750_suta M_ru_78182_suta M_ru_80819_arsu M_ru_81347_arsu M_ru_85426_arsu M_ru_85919_suta M_ru_A11834_suta M_ru_A15176_suta M_ru_A16195_pa M_ru_A5487_suta M_ru_A9235_pa M_ru_A9903_pa M_ru_J640_roar M_ru_J676_roar M_ru_T10184_suta M_ru_T11079_pa M_ru_T11238_pa M_ru_T11780_suta M_ru_T12541_pa M_ru_T14532_suta M_ru_T14622_suta M_ru_T1649_pa M_ru_T16553_pa M_ru_T19782_pa M_ru_T368_ma M_ru_T381_ma M_ru_T476_ma M_ru_T494_ma M_ru_T6500_pa M_ru_T6577_pa M_ru_T753_suta -5 0 5 10 15 PC0 (39.8%) explained -5 0 5 10 PC1 (5.6%) explained Mach Roar ArSu SuTa Para

In [30]:
pca.run_tsne(subsample=True, perplexity=5.0, n_iter=1000000, seed=123)
pca2.run_tsne(subsample=True, perplexity=5.0, n_iter=1000000, seed=123)
pca3.run_tsne(subsample=True, perplexity=5.0, n_iter=1000000, seed=123)
pca4.run_tsne(subsample=True, perplexity=5.0, n_iter=1000000, seed=123)
pca5.run_tsne(subsample=True, perplexity=5.0, n_iter=1000000, seed=123)

Subsampling SNPs: 13732/114554
Subsampling SNPs: 11711/92842
Subsampling SNPs: 12743/103830
Subsampling SNPs: 13723/114754
Subsampling SNPs: 6264/48908


In [31]:
pca.draw();
pca2.draw();
pca3.draw();
pca4.draw();
pca5.draw();

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t7ea290791b714e448347f5f6dedb8f32" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> M_ru_77750_suta M_ru_78182_suta M_ru_80819_arsu M_ru_81347_arsu M_ru_85426_arsu M_ru_85919_suta M_ru_A11834_suta M_ru_A15176_suta M_ru_A16195_pa M_ru_A5487_suta M_ru_A9235_pa M_ru_A9903_pa M_ru_J640_roar M_ru_J676_roar M_ru_T10184_suta M_ru_T11079_pa M_ru_T11238_pa M_ru_T11780_suta M_ru_T12541_pa M_ru_T14532_suta M_ru_T14622_suta M_ru_T1649_pa M_ru_T16553_pa M_ru_T19782_pa M_ru_T368_ma M_ru_T381_ma M_ru_T476_ma M_ru_T494_ma M_ru_T6500_pa M_ru_T6577_pa M_ru_T753_suta -200 0 200 400 TSNE component 1 -200 0 200 400 TSNE component 2 Mach Roar ArSu SuTa Para

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t319798a6e4064a6e8f521ba719dfb07a" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> M_ru_77750_suta M_ru_78182_suta M_ru_80819_arsu M_ru_81347_arsu M_ru_85426_arsu M_ru_85919_suta M_ru_A11834_suta M_ru_A15176_suta M_ru_A16195_pa M_ru_A5487_suta M_ru_A9235_pa M_ru_A9903_pa M_ru_J640_roar M_ru_J676_roar M_ru_T10184_suta M_ru_T11079_pa M_ru_T11238_pa M_ru_T11780_suta M_ru_T12541_pa M_ru_T14532_suta M_ru_T14622_suta M_ru_T1649_pa M_ru_T16553_pa M_ru_T19782_pa M_ru_T368_ma M_ru_T381_ma M_ru_T476_ma M_ru_T494_ma M_ru_T6500_pa M_ru_T6577_pa M_ru_T753_suta -200 0 200 400 TSNE component 1 -250 0 250 500 TSNE component 2 Mach Roar ArSu SuTa Para

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t455ad3d548bf4a8aaebef5c2e77b6e0c" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> M_ru_77750_suta M_ru_78182_suta M_ru_80819_arsu M_ru_81347_arsu M_ru_85426_arsu M_ru_85919_suta M_ru_A11834_suta M_ru_A15176_suta M_ru_A16195_pa M_ru_A5487_suta M_ru_A9235_pa M_ru_A9903_pa M_ru_J640_roar M_ru_J676_roar M_ru_T10184_suta M_ru_T11079_pa M_ru_T11238_pa M_ru_T11780_suta M_ru_T12541_pa M_ru_T14532_suta M_ru_T14622_suta M_ru_T1649_pa M_ru_T16553_pa M_ru_T19782_pa M_ru_T368_ma M_ru_T381_ma M_ru_T476_ma M_ru_T494_ma M_ru_T6500_pa M_ru_T6577_pa M_ru_T753_suta -200 -100 0 100 200 TSNE component 1 -300 0 300 600 TSNE component 2 Mach Roar ArSu SuTa Para

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t63dfde6dfa334d1696d7b6768587a663" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> M_ru_77750_suta M_ru_78182_suta M_ru_80819_arsu M_ru_81347_arsu M_ru_85426_arsu M_ru_85919_suta M_ru_A11834_suta M_ru_A15176_suta M_ru_A16195_pa M_ru_A5487_suta M_ru_A9235_pa M_ru_A9903_pa M_ru_J640_roar M_ru_J676_roar M_ru_T10184_suta M_ru_T11079_pa M_ru_T11238_pa M_ru_T11780_suta M_ru_T12541_pa M_ru_T14532_suta M_ru_T14622_suta M_ru_T1649_pa M_ru_T16553_pa M_ru_T19782_pa M_ru_T368_ma M_ru_T381_ma M_ru_T476_ma M_ru_T494_ma M_ru_T6500_pa M_ru_T6577_pa M_ru_T753_suta -400 0 400 800 TSNE component 1 -40 0 40 80 TSNE component 2 Mach Roar ArSu SuTa Para

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t222a599296c1464aa6b9445ba3bfa52c" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> M_ru_77750_suta M_ru_78182_suta M_ru_80819_arsu M_ru_81347_arsu M_ru_85426_arsu M_ru_85919_suta M_ru_A11834_suta M_ru_A15176_suta M_ru_A16195_pa M_ru_A5487_suta M_ru_A9235_pa M_ru_A9903_pa M_ru_J640_roar M_ru_J676_roar M_ru_T10184_suta M_ru_T11079_pa M_ru_T11238_pa M_ru_T11780_suta M_ru_T12541_pa M_ru_T14532_suta M_ru_T14622_suta M_ru_T1649_pa M_ru_T16553_pa M_ru_T19782_pa M_ru_T368_ma M_ru_T381_ma M_ru_T476_ma M_ru_T494_ma M_ru_T6500_pa M_ru_T6577_pa M_ru_T753_suta -250 0 250 TSNE component 1 -100 0 100 TSNE component 2 Mach Roar ArSu SuTa Para

Now let's see if the results hold up to a different clustering algorithm so I will group samples into "populations" based on results from the structure analysis to see if PCA and t-SNE look the same. Here we use "sample" to impute from a priori defined populations

In [38]:
imap = {
#"ref": ["reference"],
"Inam": ['M_ru_A7875_In','M_ru_T14456_In', 'M_ru_T23245_In', 'M_ru_T23246_In', 'M_ru_T23416_In', 'M_ru_T23478_In'],
"Puru": ['M_ru_A10311_pu','M_ru_A10329_pu', 'M_ru_A1380_pu', 'M_ru_A2741_pu', 'M_ru_A436_pu', 'M_ru_A440_pu'],
"JiGuMach": ['M_ru_A207_jigu', 'M_ru_T3228_jigu', 'M_ru_T7634_jigu','M_ru_T13253_ma','M_ru_A474_ma', 'M_ru_T3164_ma', 'M_ru_J265_ma'],
"MachRoArSuTa": ['M_ru_T368_ma', 'M_ru_T381_ma', 'M_ru_T476_ma', 'M_ru_T494_ma','M_ru_J640_roar','M_ru_J676_roar','M_ru_80819_arsu', 'M_ru_81347_arsu', 'M_ru_85426_arsu','M_ru_85919_suta','M_ru_77750_suta', 'M_ru_78182_suta', 'M_ru_A11834_suta', 'M_ru_A15176_suta', 'M_ru_A5487_suta', 'M_ru_T10184_suta', 'M_ru_T11780_suta', 'M_ru_T14532_suta', 'M_ru_T14622_suta', 'M_ru_T753_suta', 'M_ru_A9903_pa'],
"Para": ['M_ru_A16195_pa','M_ru_A9235_pa', 'M_ru_T11079_pa', 'M_ru_T11238_pa', 'M_ru_T12541_pa', 'M_ru_T1649_pa', 'M_ru_T16553_pa', 'M_ru_T19782_pa', 'M_ru_T6500_pa', 'M_ru_T6577_pa']
}

# minimum % of samples that must be present in each SNP from each group
minmap1 = {i: 0.55 for i in imap}
minmap2 = {i: 0.65 for i in imap}
minmap3 = {i: 0.75 for i in imap}
minmap4 = {i: 0.85 for i in imap}
minmap5 = {i: 0.95 for i in imap}

In [39]:
# init pca object with input data and (optional) parameter options
pca = ipa.pca(
    data=data,
    imap=imap,
    minmap=minmap1,
    mincov=0.85,
    impute_method="sample",
)
# init pca object with input data and (optional) parameter options
pca2 = ipa.pca(
    data=data,
    imap=imap,
    minmap=minmap2,
    mincov=0.85,
    impute_method="sample",
)
# init pca object with input data and (optional) parameter options
pca3 = ipa.pca(
    data=data,
    imap=imap,
    minmap=minmap3,
    mincov=0.85,
    impute_method="sample",
)
# init pca object with input data and (optional) parameter options
pca4 = ipa.pca(
    data=data,
    imap=imap,
    minmap=minmap4,
    mincov=0.85,
    impute_method="sample",
)
# init pca object with input data and (optional) parameter options
pca5 = ipa.pca(
    data=data,
    imap=imap,
    minmap=minmap5,
    mincov=0.85,
    impute_method="sample",
)

Samples: 50
Sites before filtering: 819934
Filtered (indels): 0
Filtered (bi-allel): 14244
Filtered (mincov): 666808
Filtered (minmap): 623187
Filtered (combined): 675486
Sites after filtering: 144448
Sites containing missing values: 111860 (77.44%)
Missing values in SNP matrix: 369541 (5.12%)
Imputation: 'sampled'; (0, 1, 2) = 36.4%, 2.1%, 61.5%
Samples: 50
Sites before filtering: 819934
Filtered (indels): 0
Filtered (bi-allel): 14244
Filtered (mincov): 666808
Filtered (minmap): 643984
Filtered (combined): 678585
Sites after filtering: 141349
Sites containing missing values: 108761 (76.95%)
Missing values in SNP matrix: 352383 (4.99%)
Imputation: 'sampled'; (0, 1, 2) = 36.5%, 2.1%, 61.5%
Samples: 50
Sites before filtering: 819934
Filtered (indels): 0
Filtered (bi-allel): 14244
Filtered (mincov): 666808
Filtered (minmap): 700135
Filtered (combined): 705183
Sites after filtering: 114751
Sites containing missing values: 82163 (71.60%)
Missing values in SNP matrix: 217012 (3.78%)
Imputati

In [40]:
# run the PCA analysis
pca.run()
pca2.run()
pca3.run()
pca4.run()
pca5.run()

Subsampling SNPs: 16483/144448
Subsampling SNPs: 16134/141349
Subsampling SNPs: 13694/114751
Subsampling SNPs: 9807/78774
Subsampling SNPs: 5224/39369


In [41]:
pca.draw();
pca2.draw();
pca3.draw();
pca4.draw();
pca5.draw();

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t52d7cdb180e8455ab9179afb9d7dda07" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> M_ru_77750_suta M_ru_78182_suta M_ru_80819_arsu M_ru_81347_arsu M_ru_85426_arsu M_ru_85919_suta M_ru_A10311_pu M_ru_A10329_pu M_ru_A11834_suta M_ru_A1380_pu M_ru_A15176_suta M_ru_A16195_pa M_ru_A207_jigu M_ru_A2741_pu M_ru_A436_pu M_ru_A440_pu M_ru_A474_ma M_ru_A5487_suta M_ru_A7875_In M_ru_A9235_pa M_ru_A9903_pa M_ru_J265_ma M_ru_J640_roar M_ru_J676_roar M_ru_T10184_suta M_ru_T11079_pa M_ru_T11238_pa M_ru_T11780_suta M_ru_T12541_pa M_ru_T13253_ma M_ru_T14456_In M_ru_T14532_suta M_ru_T14622_suta M_ru_T1649_pa M_ru_T16553_pa M_ru_T19782_pa M_ru_T23245_In M_ru_T23246_In M_ru_T23416_In M_ru_T23478_In M_ru_T3164_ma M_ru_T3228_jigu M_ru_T368_ma M_ru_T381_ma M_ru_T476_ma M_ru_T494_ma M_ru_T6500_pa M_ru_T6577_pa M_ru_T753_suta M_ru_T7634_jigu -10 0 10 20 PC0 (28.1%) explained -10 0 10 20 PC1 (17.2%) explained Inam Puru JiGuMach MachRoArSuTa Para

<svg class="toyplot-canvas-Canvas" height="300.0px" id="tfdf52519ea8b46d5b9376a248ca81a8c" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> M_ru_77750_suta M_ru_78182_suta M_ru_80819_arsu M_ru_81347_arsu M_ru_85426_arsu M_ru_85919_suta M_ru_A10311_pu M_ru_A10329_pu M_ru_A11834_suta M_ru_A1380_pu M_ru_A15176_suta M_ru_A16195_pa M_ru_A207_jigu M_ru_A2741_pu M_ru_A436_pu M_ru_A440_pu M_ru_A474_ma M_ru_A5487_suta M_ru_A7875_In M_ru_A9235_pa M_ru_A9903_pa M_ru_J265_ma M_ru_J640_roar M_ru_J676_roar M_ru_T10184_suta M_ru_T11079_pa M_ru_T11238_pa M_ru_T11780_suta M_ru_T12541_pa M_ru_T13253_ma M_ru_T14456_In M_ru_T14532_suta M_ru_T14622_suta M_ru_T1649_pa M_ru_T16553_pa M_ru_T19782_pa M_ru_T23245_In M_ru_T23246_In M_ru_T23416_In M_ru_T23478_In M_ru_T3164_ma M_ru_T3228_jigu M_ru_T368_ma M_ru_T381_ma M_ru_T476_ma M_ru_T494_ma M_ru_T6500_pa M_ru_T6577_pa M_ru_T753_suta M_ru_T7634_jigu -10 0 10 20 PC0 (28.0%) explained -10 0 10 20 PC1 (18.2%) explained Inam Puru JiGuMach MachRoArSuTa Para

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t8668d33f1453493aac33439fad3675a6" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> M_ru_77750_suta M_ru_78182_suta M_ru_80819_arsu M_ru_81347_arsu M_ru_85426_arsu M_ru_85919_suta M_ru_A10311_pu M_ru_A10329_pu M_ru_A11834_suta M_ru_A1380_pu M_ru_A15176_suta M_ru_A16195_pa M_ru_A207_jigu M_ru_A2741_pu M_ru_A436_pu M_ru_A440_pu M_ru_A474_ma M_ru_A5487_suta M_ru_A7875_In M_ru_A9235_pa M_ru_A9903_pa M_ru_J265_ma M_ru_J640_roar M_ru_J676_roar M_ru_T10184_suta M_ru_T11079_pa M_ru_T11238_pa M_ru_T11780_suta M_ru_T12541_pa M_ru_T13253_ma M_ru_T14456_In M_ru_T14532_suta M_ru_T14622_suta M_ru_T1649_pa M_ru_T16553_pa M_ru_T19782_pa M_ru_T23245_In M_ru_T23246_In M_ru_T23416_In M_ru_T23478_In M_ru_T3164_ma M_ru_T3228_jigu M_ru_T368_ma M_ru_T381_ma M_ru_T476_ma M_ru_T494_ma M_ru_T6500_pa M_ru_T6577_pa M_ru_T753_suta M_ru_T7634_jigu -10 0 10 20 PC0 (27.5%) explained -10 0 10 20 PC1 (18.9%) explained Inam Puru JiGuMach MachRoArSuTa Para

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t6568ba4810dc4237b43478d6ff35e96c" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> M_ru_77750_suta M_ru_78182_suta M_ru_80819_arsu M_ru_81347_arsu M_ru_85426_arsu M_ru_85919_suta M_ru_A10311_pu M_ru_A10329_pu M_ru_A11834_suta M_ru_A1380_pu M_ru_A15176_suta M_ru_A16195_pa M_ru_A207_jigu M_ru_A2741_pu M_ru_A436_pu M_ru_A440_pu M_ru_A474_ma M_ru_A5487_suta M_ru_A7875_In M_ru_A9235_pa M_ru_A9903_pa M_ru_J265_ma M_ru_J640_roar M_ru_J676_roar M_ru_T10184_suta M_ru_T11079_pa M_ru_T11238_pa M_ru_T11780_suta M_ru_T12541_pa M_ru_T13253_ma M_ru_T14456_In M_ru_T14532_suta M_ru_T14622_suta M_ru_T1649_pa M_ru_T16553_pa M_ru_T19782_pa M_ru_T23245_In M_ru_T23246_In M_ru_T23416_In M_ru_T23478_In M_ru_T3164_ma M_ru_T3228_jigu M_ru_T368_ma M_ru_T381_ma M_ru_T476_ma M_ru_T494_ma M_ru_T6500_pa M_ru_T6577_pa M_ru_T753_suta M_ru_T7634_jigu -10 0 10 20 PC0 (27.5%) explained -10 0 10 20 PC1 (18.6%) explained Inam Puru JiGuMach MachRoArSuTa Para

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t60cec539cbf845abaed86fd53d914306" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> M_ru_77750_suta M_ru_78182_suta M_ru_80819_arsu M_ru_81347_arsu M_ru_85426_arsu M_ru_85919_suta M_ru_A10311_pu M_ru_A10329_pu M_ru_A11834_suta M_ru_A1380_pu M_ru_A15176_suta M_ru_A16195_pa M_ru_A207_jigu M_ru_A2741_pu M_ru_A436_pu M_ru_A440_pu M_ru_A474_ma M_ru_A5487_suta M_ru_A7875_In M_ru_A9235_pa M_ru_A9903_pa M_ru_J265_ma M_ru_J640_roar M_ru_J676_roar M_ru_T10184_suta M_ru_T11079_pa M_ru_T11238_pa M_ru_T11780_suta M_ru_T12541_pa M_ru_T13253_ma M_ru_T14456_In M_ru_T14532_suta M_ru_T14622_suta M_ru_T1649_pa M_ru_T16553_pa M_ru_T19782_pa M_ru_T23245_In M_ru_T23246_In M_ru_T23416_In M_ru_T23478_In M_ru_T3164_ma M_ru_T3228_jigu M_ru_T368_ma M_ru_T381_ma M_ru_T476_ma M_ru_T494_ma M_ru_T6500_pa M_ru_T6577_pa M_ru_T753_suta M_ru_T7634_jigu -10 0 10 PC0 (27.5%) explained -5 0 5 10 15 PC1 (18.3%) explained Inam Puru JiGuMach MachRoArSuTa Para

In [44]:
pca.run_tsne(subsample=True, perplexity=4.0, n_iter=1000000, seed=223)
pca2.run_tsne(subsample=True, perplexity=4.0, n_iter=1000000, seed=123)
pca3.run_tsne(subsample=True, perplexity=4.0, n_iter=1000000, seed=223)
pca4.run_tsne(subsample=True, perplexity=4.0, n_iter=1000000, seed=123)
pca5.run_tsne(subsample=True, perplexity=4.0, n_iter=1000000, seed=223)

Subsampling SNPs: 16483/144448
Subsampling SNPs: 16134/141349
Subsampling SNPs: 13694/114751
Subsampling SNPs: 9807/78774
Subsampling SNPs: 5224/39369


In [45]:
pca.draw();
pca2.draw();
pca3.draw();
pca4.draw();
pca5.draw();

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t2cd7a1ebf1cc409298178013b0bb996e" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> M_ru_77750_suta M_ru_78182_suta M_ru_80819_arsu M_ru_81347_arsu M_ru_85426_arsu M_ru_85919_suta M_ru_A10311_pu M_ru_A10329_pu M_ru_A11834_suta M_ru_A1380_pu M_ru_A15176_suta M_ru_A16195_pa M_ru_A207_jigu M_ru_A2741_pu M_ru_A436_pu M_ru_A440_pu M_ru_A474_ma M_ru_A5487_suta M_ru_A7875_In M_ru_A9235_pa M_ru_A9903_pa M_ru_J265_ma M_ru_J640_roar M_ru_J676_roar M_ru_T10184_suta M_ru_T11079_pa M_ru_T11238_pa M_ru_T11780_suta M_ru_T12541_pa M_ru_T13253_ma M_ru_T14456_In M_ru_T14532_suta M_ru_T14622_suta M_ru_T1649_pa M_ru_T16553_pa M_ru_T19782_pa M_ru_T23245_In M_ru_T23246_In M_ru_T23416_In M_ru_T23478_In M_ru_T3164_ma M_ru_T3228_jigu M_ru_T368_ma M_ru_T381_ma M_ru_T476_ma M_ru_T494_ma M_ru_T6500_pa M_ru_T6577_pa M_ru_T753_suta M_ru_T7634_jigu -1000 -500 0 500 1000 TSNE component 1 -500 -250 0 250 TSNE component 2 Inam Puru JiGuMach MachRoArSuTa Para

<svg class="toyplot-canvas-Canvas" height="300.0px" id="tf2ef56da2db8465b9266151a0b5b68b1" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> M_ru_77750_suta M_ru_78182_suta M_ru_80819_arsu M_ru_81347_arsu M_ru_85426_arsu M_ru_85919_suta M_ru_A10311_pu M_ru_A10329_pu M_ru_A11834_suta M_ru_A1380_pu M_ru_A15176_suta M_ru_A16195_pa M_ru_A207_jigu M_ru_A2741_pu M_ru_A436_pu M_ru_A440_pu M_ru_A474_ma M_ru_A5487_suta M_ru_A7875_In M_ru_A9235_pa M_ru_A9903_pa M_ru_J265_ma M_ru_J640_roar M_ru_J676_roar M_ru_T10184_suta M_ru_T11079_pa M_ru_T11238_pa M_ru_T11780_suta M_ru_T12541_pa M_ru_T13253_ma M_ru_T14456_In M_ru_T14532_suta M_ru_T14622_suta M_ru_T1649_pa M_ru_T16553_pa M_ru_T19782_pa M_ru_T23245_In M_ru_T23246_In M_ru_T23416_In M_ru_T23478_In M_ru_T3164_ma M_ru_T3228_jigu M_ru_T368_ma M_ru_T381_ma M_ru_T476_ma M_ru_T494_ma M_ru_T6500_pa M_ru_T6577_pa M_ru_T753_suta M_ru_T7634_jigu -2500 0 2500 TSNE component 1 -2000 -1000 0 1000 2000 TSNE component 2 Inam Puru JiGuMach MachRoArSuTa Para

<svg class="toyplot-canvas-Canvas" height="300.0px" id="tddeea1c0c40a4814ac7d864622c467f6" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> M_ru_77750_suta M_ru_78182_suta M_ru_80819_arsu M_ru_81347_arsu M_ru_85426_arsu M_ru_85919_suta M_ru_A10311_pu M_ru_A10329_pu M_ru_A11834_suta M_ru_A1380_pu M_ru_A15176_suta M_ru_A16195_pa M_ru_A207_jigu M_ru_A2741_pu M_ru_A436_pu M_ru_A440_pu M_ru_A474_ma M_ru_A5487_suta M_ru_A7875_In M_ru_A9235_pa M_ru_A9903_pa M_ru_J265_ma M_ru_J640_roar M_ru_J676_roar M_ru_T10184_suta M_ru_T11079_pa M_ru_T11238_pa M_ru_T11780_suta M_ru_T12541_pa M_ru_T13253_ma M_ru_T14456_In M_ru_T14532_suta M_ru_T14622_suta M_ru_T1649_pa M_ru_T16553_pa M_ru_T19782_pa M_ru_T23245_In M_ru_T23246_In M_ru_T23416_In M_ru_T23478_In M_ru_T3164_ma M_ru_T3228_jigu M_ru_T368_ma M_ru_T381_ma M_ru_T476_ma M_ru_T494_ma M_ru_T6500_pa M_ru_T6577_pa M_ru_T753_suta M_ru_T7634_jigu -1000 -500 0 500 TSNE component 1 -2000 -1000 0 1000 TSNE component 2 Inam Puru JiGuMach MachRoArSuTa Para

<svg class="toyplot-canvas-Canvas" height="300.0px" id="tbc0831cf76864ba38622277a1c7b275f" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> M_ru_77750_suta M_ru_78182_suta M_ru_80819_arsu M_ru_81347_arsu M_ru_85426_arsu M_ru_85919_suta M_ru_A10311_pu M_ru_A10329_pu M_ru_A11834_suta M_ru_A1380_pu M_ru_A15176_suta M_ru_A16195_pa M_ru_A207_jigu M_ru_A2741_pu M_ru_A436_pu M_ru_A440_pu M_ru_A474_ma M_ru_A5487_suta M_ru_A7875_In M_ru_A9235_pa M_ru_A9903_pa M_ru_J265_ma M_ru_J640_roar M_ru_J676_roar M_ru_T10184_suta M_ru_T11079_pa M_ru_T11238_pa M_ru_T11780_suta M_ru_T12541_pa M_ru_T13253_ma M_ru_T14456_In M_ru_T14532_suta M_ru_T14622_suta M_ru_T1649_pa M_ru_T16553_pa M_ru_T19782_pa M_ru_T23245_In M_ru_T23246_In M_ru_T23416_In M_ru_T23478_In M_ru_T3164_ma M_ru_T3228_jigu M_ru_T368_ma M_ru_T381_ma M_ru_T476_ma M_ru_T494_ma M_ru_T6500_pa M_ru_T6577_pa M_ru_T753_suta M_ru_T7634_jigu -4000 -2000 0 2000 TSNE component 1 -2000 -1000 0 1000 TSNE component 2 Inam Puru JiGuMach MachRoArSuTa Para

<svg class="toyplot-canvas-Canvas" height="300.0px" id="tada40cc757564615b4bb4c85b4676eb6" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> M_ru_77750_suta M_ru_78182_suta M_ru_80819_arsu M_ru_81347_arsu M_ru_85426_arsu M_ru_85919_suta M_ru_A10311_pu M_ru_A10329_pu M_ru_A11834_suta M_ru_A1380_pu M_ru_A15176_suta M_ru_A16195_pa M_ru_A207_jigu M_ru_A2741_pu M_ru_A436_pu M_ru_A440_pu M_ru_A474_ma M_ru_A5487_suta M_ru_A7875_In M_ru_A9235_pa M_ru_A9903_pa M_ru_J265_ma M_ru_J640_roar M_ru_J676_roar M_ru_T10184_suta M_ru_T11079_pa M_ru_T11238_pa M_ru_T11780_suta M_ru_T12541_pa M_ru_T13253_ma M_ru_T14456_In M_ru_T14532_suta M_ru_T14622_suta M_ru_T1649_pa M_ru_T16553_pa M_ru_T19782_pa M_ru_T23245_In M_ru_T23246_In M_ru_T23416_In M_ru_T23478_In M_ru_T3164_ma M_ru_T3228_jigu M_ru_T368_ma M_ru_T381_ma M_ru_T476_ma M_ru_T494_ma M_ru_T6500_pa M_ru_T6577_pa M_ru_T753_suta M_ru_T7634_jigu -200 -100 0 100 200 TSNE component 1 -300 -200 -100 0 100 200 TSNE component 2 Inam Puru JiGuMach MachRoArSuTa Para

Amazingly, the results look rather similar to the initial results of imputation by popuilations defined by the k-means clustering analysis

(You can place your cursor on a point to see the label)

# Genetic Distances (dxy)

Now let's generate distance matrices for EEMs and see if imputation effects measures of distance. First look at imputation results based on structure assignments, and then look at imputation results based on a priori assignments

In [46]:
imap = {
#"ref": ["reference"],
"Inam": ['M_ru_A7875_In','M_ru_T14456_In', 'M_ru_T23245_In', 'M_ru_T23246_In', 'M_ru_T23416_In', 'M_ru_T23478_In'],
"Puru": ['M_ru_A10311_pu','M_ru_A10329_pu', 'M_ru_A1380_pu', 'M_ru_A2741_pu', 'M_ru_A436_pu', 'M_ru_A440_pu'],
"JiGuMach": ['M_ru_A207_jigu', 'M_ru_T3228_jigu', 'M_ru_T7634_jigu','M_ru_T13253_ma','M_ru_A474_ma', 'M_ru_T3164_ma', 'M_ru_J265_ma'],
"MachRoArSuTa": ['M_ru_T368_ma', 'M_ru_T381_ma', 'M_ru_T476_ma', 'M_ru_T494_ma','M_ru_J640_roar','M_ru_J676_roar','M_ru_80819_arsu', 'M_ru_81347_arsu', 'M_ru_85426_arsu','M_ru_85919_suta','M_ru_77750_suta', 'M_ru_78182_suta', 'M_ru_A11834_suta', 'M_ru_A15176_suta', 'M_ru_A5487_suta', 'M_ru_T10184_suta', 'M_ru_T11780_suta', 'M_ru_T14532_suta', 'M_ru_T14622_suta', 'M_ru_T753_suta', 'M_ru_A9903_pa'],
"Para": ['M_ru_A16195_pa','M_ru_A9235_pa', 'M_ru_T11079_pa', 'M_ru_T11238_pa', 'M_ru_T12541_pa', 'M_ru_T1649_pa', 'M_ru_T16553_pa', 'M_ru_T19782_pa', 'M_ru_T6500_pa', 'M_ru_T6577_pa']
}

minmap4 = {i: 0.85 for i in imap}


In [47]:
# load the snp data into distance tool with arguments
from ipyrad.analysis.distance import Distance
dist = Distance(
    data=data, 
    imap=imap,
    minmap=minmap4,
    mincov=0.85,
    impute_method="sample",
    subsample_snps=False,
)
dist.run()

Samples: 50
Sites before filtering: 819934
Filtered (indels): 0
Filtered (bi-allel): 14244
Filtered (mincov): 666808
Filtered (minmap): 739034
Filtered (combined): 741160
Sites after filtering: 78774
Sites containing missing values: 46186 (58.63%)
Missing values in SNP matrix: 82019 (2.08%)
Imputation: 'sampled'; (0, 1, 2) = 37.1%, 2.1%, 60.8%


In [48]:
# save to a CSV file
dist.dists.to_csv("M_ru_distances_12Jan2022.csv")

# save to a CSV file with no labels (eems style)
dist.dists.to_csv(
    "M_ru_distances_eems_12Jan2022.csv",
    header=None,
    index=False,
    sep=" ",
)

In [49]:
imap = {
#"ref": ["reference"],
"Inam": ['M_ru_A7875_In','M_ru_T14456_In', 'M_ru_T23245_In', 'M_ru_T23246_In', 'M_ru_T23416_In', 'M_ru_T23478_In'],
"Puru": ['M_ru_A10311_pu','M_ru_A10329_pu', 'M_ru_A1380_pu', 'M_ru_A2741_pu', 'M_ru_A436_pu', 'M_ru_A440_pu'],
"JiGu": ['M_ru_A207_jigu', 'M_ru_T3228_jigu', 'M_ru_T7634_jigu'],
"Mach": ['M_ru_T13253_ma','M_ru_A474_ma', 'M_ru_T3164_ma', 'M_ru_J265_ma', 'M_ru_T368_ma', 'M_ru_T381_ma', 'M_ru_T476_ma', 'M_ru_T494_ma'],
"Roar": ['M_ru_J640_roar','M_ru_J676_roar'],
"ArSu": ['M_ru_80819_arsu', 'M_ru_81347_arsu', 'M_ru_85426_arsu'],
"SuTa": [ 'M_ru_85919_suta','M_ru_77750_suta', 'M_ru_78182_suta', 'M_ru_A11834_suta', 'M_ru_A15176_suta', 'M_ru_A5487_suta', 'M_ru_T10184_suta', 'M_ru_T11780_suta', 'M_ru_T14532_suta', 'M_ru_T14622_suta', 'M_ru_T753_suta', 'M_ru_A9903_pa'],
"Para": ['M_ru_A16195_pa','M_ru_A9235_pa', 'M_ru_T11079_pa', 'M_ru_T11238_pa', 'M_ru_T12541_pa', 'M_ru_T1649_pa', 'M_ru_T16553_pa', 'M_ru_T19782_pa', 'M_ru_T6500_pa', 'M_ru_T6577_pa']
}

minmap4 = {i: 0.85 for i in imap}

In [50]:
# load the snp data into distance tool with arguments
from ipyrad.analysis.distance import Distance
dist2 = Distance(
    data=data, 
    imap=imap,
    minmap=minmap4,
    mincov=0.85,
    impute_method="sample",
    subsample_snps=False,
)
dist2.run()

Samples: 50
Sites before filtering: 819934
Filtered (indels): 0
Filtered (bi-allel): 14244
Filtered (mincov): 666808
Filtered (minmap): 760928
Filtered (combined): 762475
Sites after filtering: 57459
Sites containing missing values: 24871 (43.28%)
Missing values in SNP matrix: 32039 (1.12%)
Imputation: 'sampled'; (0, 1, 2) = 37.3%, 2.0%, 60.6%


In [51]:
# get list of concatenated names from each group
ordered_names = []
for group in dist.imap.values():
    ordered_names += group

# reorder matrix to match name order    
ordered_matrix = dist.dists[ordered_names].T[ordered_names]

toyplot.matrix(
    ordered_matrix,
    bshow=False,
    tshow=False,
    rlocator=toyplot.locator.Explicit(
        range(len(ordered_names)),
        ordered_names,
));

# get list of concatenated names from each group
ordered_names = []
for group in dist2.imap.values():
    ordered_names += group

# reorder matrix to match name order    
ordered_matrix = dist2.dists[ordered_names].T[ordered_names]

toyplot.matrix(
    ordered_matrix,
    bshow=False,
    tshow=False,
    rlocator=toyplot.locator.Explicit(
        range(len(ordered_names)),
        ordered_names,
));

<svg class="toyplot-canvas-Canvas" height="600px" id="t1c39d92d144a40529f0ba770fec6ed88" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 600 600" width="600px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 0 0.000000 0.033336 0.029591 0.030061 0.031939 0.031889 0.089915 0.091299 0.091578 0.090639 0.092061 0.090004 0.094816 0.093280 0.094105 0.089446 0.090233 0.090398 0.089966 0.122959 0.122045 0.121093 0.122553 0.126133 0.125943 0.126108 0.126514 0.124445 0.126044 0.126958 0.126996 0.125981 0.125232 0.126514 0.126133 0.126412 0.126438 0.125359 0.125549 0.127123 0.134016 0.134918 0.134499 0.135298 0.132848 0.134410 0.134092 0.135387 0.135121 0.135146 M_ru_A7875_In 1 0.033336 0.000000 0.031571 0.030695 0.028283 0.028309 0.088925 0.090741 0.091604 0.090588 0.091782 0.089877 0.094663 0.093863 0.093876 0.089725 0.089852 0.090550 0.090042 0.123416 0.122883 0.121804 0.123188 0.126565 0.126222 0.126615 0.127377 0.125003 0.127237 0.127339 0.127733 0.127174 0.125663 0.127326 0.127225 0.127047 0.127504 0.125790 0.126158 0.127529 0.133788 0.134740 0.134600 0.135527 0.132671 0.134410 0.134143 0.135667 0.134816 0.134943 M_ru_T14456_In 2 0.029591 0.031571 0.000000 0.029210 0.029464 0.029997 0.087872 0.089332 0.090042 0.089154 0.090576 0.088773 0.093356 0.092277 0.092594 0.088265 0.088697 0.089395 0.088760 0.122540 0.121626 0.120471 0.121601 0.125003 0.124940 0.124825 0.125739 0.123518 0.125346 0.125447 0.126171 0.125562 0.124280 0.125841 0.125613 0.125206 0.125638 0.124559 0.124343 0.125765 0.132734 0.133915 0.133572 0.134575 0.131389 0.133458 0.133343 0.134435 0.134092 0.133839 M_ru_T23245_In 3 0.030061 0.030695 0.029210 0.000000 0.029959 0.028690 0.087300 0.088913 0.089979 0.088659 0.089954 0.087897 0.092784 0.091477 0.091870 0.087034 0.087694 0.088646 0.088062 0.121182 0.120268 0.119773 0.120751 0.124051 0.123784 0.123924 0.124711 0.122388 0.124419 0.124749 0.125219 0.124533 0.123353 0.124407 0.124864 0.124432 0.124711 0.123302 0.123264 0.124762 0.131452 0.132429 0.132366 0.133166 0.130157 0.131871 0.131731 0.133635 0.132836 0.132506 M_ru_T23246_In 4 0.031939 0.028283 0.029464 0.029959 0.000000 0.025567 0.087732 0.089598 0.090563 0.089192 0.090994 0.088836 0.093521 0.092518 0.092784 0.088100 0.089090 0.089738 0.088621 0.122223 0.121766 0.120078 0.121411 0.125574 0.125181 0.124965 0.125600 0.123505 0.125866 0.125892 0.126311 0.125600 0.123937 0.125828 0.125803 0.125143 0.125651 0.124419 0.124635 0.125447 0.133077 0.134308 0.133889 0.134867 0.131477 0.133750 0.133382 0.134753 0.134232 0.134054 M_ru_T23416_In 5 0.031889 0.028309 0.029997 0.028690 0.025567 0.000000 0.088214 0.089954 0.090791 0.089522 0.091299 0.088938 0.093495 0.092645 0.092911 0.088456 0.088786 0.089662 0.088925 0.122248 0.121791 0.120408 0.121918 0.125346 0.124902 0.125168 0.125904 0.123632 0.125739 0.126095 0.126158 0.125600 0.124140 0.125752 0.125904 0.125371 0.125701 0.124242 0.124762 0.125701 0.132391 0.133293 0.133305 0.134257 0.131300 0.133191 0.132544 0.134423 0.133724 0.133597 M_ru_T23478_In 6 0.089915 0.088925 0.087872 0.087300 0.087732 0.088214 0.000000 0.039036 0.041473 0.038833 0.039823 0.040153 0.064793 0.064526 0.064057 0.064399 0.068716 0.068322 0.068627 0.112258 0.111115 0.110392 0.111826 0.116802 0.117069 0.117640 0.117970 0.116130 0.118313 0.118770 0.118351 0.118478 0.116688 0.117818 0.118427 0.118148 0.117793 0.116891 0.116904 0.118072 0.125270 0.126120 0.126031 0.127034 0.124254 0.125435 0.125600 0.127733 0.126273 0.126704 M_ru_A10311_pu 7 0.091299 0.090741 0.089332 0.088913 0.089598 0.089954 0.039036 0.000000 0.041714 0.040800 0.040318 0.040953 0.066126 0.065631 0.064907 0.064717 0.068804 0.068411 0.068868 0.111890 0.110950 0.

<svg class="toyplot-canvas-Canvas" height="600px" id="tdfb303144f5c41ce9d34192d91b4345d" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 600 600" width="600px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 0 0.000000 0.033311 0.029343 0.029325 0.030857 0.031866 0.088550 0.089525 0.089734 0.089351 0.092100 0.089716 0.093145 0.092779 0.093441 0.087436 0.089316 0.090377 0.088602 0.117597 0.116657 0.117527 0.118397 0.121269 0.120991 0.120260 0.122157 0.119459 0.121878 0.121722 0.122244 0.120677 0.120312 0.121704 0.121461 0.121199 0.121635 0.120956 0.120799 0.121582 0.131659 0.132669 0.133191 0.133243 0.130145 0.132251 0.131833 0.132895 0.132808 0.133278 M_ru_A7875_In 1 0.033311 0.000000 0.031222 0.030857 0.027689 0.028455 0.087541 0.089107 0.089629 0.089107 0.091822 0.089507 0.092692 0.093267 0.092988 0.087471 0.088620 0.090586 0.088498 0.117771 0.116901 0.117771 0.118780 0.121234 0.120782 0.120538 0.122644 0.119633 0.122574 0.121617 0.122348 0.121408 0.120103 0.121878 0.121878 0.121478 0.122296 0.120921 0.121008 0.121548 0.131381 0.132216 0.132877 0.133208 0.129901 0.131868 0.131520 0.132756 0.132425 0.132895 M_ru_T14456_In 2 0.029343 0.031222 0.000000 0.028873 0.028559 0.030021 0.085940 0.087297 0.087715 0.087123 0.090256 0.087976 0.091404 0.091422 0.091457 0.085661 0.086984 0.088185 0.086375 0.116553 0.115543 0.116344 0.116936 0.119598 0.119598 0.118554 0.120869 0.118032 0.120590 0.119842 0.120956 0.119668 0.118850 0.120451 0.120312 0.119529 0.120416 0.119633 0.118850 0.119738 0.130267 0.131241 0.131868 0.132338 0.128544 0.130998 0.130684 0.131885 0.131520 0.131781 M_ru_T23245_In 3 0.029325 0.030857 0.028873 0.000000 0.029726 0.029029 0.085818 0.087245 0.088289 0.087210 0.089890 0.087575 0.090865 0.090708 0.090778 0.084739 0.086235 0.087924 0.085905 0.115387 0.114342 0.115770 0.116257 0.118746 0.118641 0.117667 0.119877 0.117075 0.119981 0.119372 0.120068 0.118815 0.117997 0.119146 0.119703 0.118920 0.119494 0.118572 0.117997 0.118711 0.128787 0.129762 0.130597 0.130859 0.127308 0.129310 0.129136 0.130580 0.130215 0.130267 M_ru_T23246_In 4 0.030857 0.027689 0.028559 0.029726 0.000000 0.025235 0.084669 0.086618 0.087558 0.086270 0.089507 0.086914 0.090169 0.090639 0.090639 0.085017 0.086618 0.087889 0.085591 0.115387 0.114586 0.114899 0.115804 0.119059 0.118606 0.117771 0.119668 0.117040 0.120051 0.119302 0.119964 0.118780 0.117371 0.119424 0.119668 0.118572 0.119355 0.118572 0.118380 0.118502 0.129344 0.130528 0.130980 0.131346 0.127343 0.130075 0.129518 0.130615 0.130563 0.130754 M_ru_T23416_In 5 0.031866 0.028455 0.030021 0.029029 0.025235 0.000000 0.085922 0.087663 0.088359 0.087280 0.090552 0.087715 0.090691 0.091335 0.091195 0.085574 0.086827 0.088376 0.086636 0.115804 0.115247 0.115735 0.116779 0.119476 0.118954 0.118537 0.120434 0.117841 0.120677 0.119964 0.120312 0.119198 0.118067 0.119842 0.120086 0.119372 0.119981 0.118815 0.118937 0.119268 0.129658 0.130458 0.131189 0.131485 0.128109 0.130354 0.129518 0.131241 0.130911 0.131137 M_ru_T23478_In 6 0.088550 0.087541 0.085940 0.085818 0.084669 0.085922 0.000000 0.037975 0.039785 0.038114 0.039263 0.039593 0.062967 0.063019 0.063054 0.063315 0.067318 0.068101 0.067509 0.106319 0.104892 0.106250 0.107329 0.111523 0.111836 0.111419 0.113142 0.110653 0.113350 0.113368 0.112950 0.112637 0.111018 0.112341 0.112968 0.112463 0.112724 0.112254 0.112028 0.111975 0.121878 0.122644 0.123862 0.123740 0.120364 0.122157 0.122191 0.123845 0.122748 0.123218 M_ru_A10311_pu 7 0.089525 0.089107 0.087297 0.087245 0.086618 0.087663 0.037975 0.000000 0.040585 0.040307 0.040864 0.041195 0.064533 0.064794 0.064516 0.063941 0.067944 0.068866 0.067509 0.107329 0.105902 0.

top=structure assignments, bottom=a priori assignments and you can see they are nearly identical